# Facebook Auction: Human or Bot?
- https://www.kaggle.com/c/facebook-recruiting-iv-human-or-bot/overview

# Credit

This notebook is inspired by great approaches sharing on Facebook Auction competition:
- https://www.kaggle.com/c/facebook-recruiting-iv-human-or-bot/discussion/14628

# Date
- Last modified: 20/05/2020

# Library

In [1]:
# Data Processing
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from scipy.stats import norm
from scipy import stats

# Data Modelling
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import xgboost as xgb
import lightgbm as lgb
from sklearn import metrics

# Exploration

In [2]:
# Importing the dataset
bids = pd.read_csv('bids.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
pd.set_option('display.max_columns', None)

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2013 entries, 0 to 2012
Data columns (total 4 columns):
bidder_id          2013 non-null object
payment_account    2013 non-null object
address            2013 non-null object
outcome            2013 non-null float64
dtypes: float64(1), object(3)
memory usage: 63.0+ KB


In [4]:
train['address'].describe()

count                                      2013
unique                                     2013
top       a3d2de7675556553a5f08e4c88d2c2285n0en
freq                                          1
Name: address, dtype: object

In [5]:
train['payment_account'].describe()

count                                      2013
unique                                     2013
top       a3d2de7675556553a5f08e4c88d2c22881lp9
freq                                          1
Name: payment_account, dtype: object

In [6]:
train_n_test = pd.concat([train, test], axis=0, sort=False)

In [7]:
train_n_test = train_n_test.reset_index()

In [8]:
train_n_test['address'].describe()

count                                      6713
unique                                     6713
top       f85f82a185b9475e497f17ebc3f5134256wr6
freq                                          1
Name: address, dtype: object

In [9]:
bids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7656334 entries, 0 to 7656333
Data columns (total 9 columns):
bid_id         int64
bidder_id      object
auction        object
merchandise    object
device         object
time           int64
country        object
ip             object
url            object
dtypes: int64(2), object(7)
memory usage: 525.7+ MB


In [10]:
bids['auction'].describe()

count     7656334
unique      15051
top         jqx39
freq       537347
Name: auction, dtype: object

In [11]:
bids.head(10)

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,9759243157894736,py,112.54.208.157,vasstdc27m7nks3
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,9759243157894736,in,18.99.175.133,vasstdc27m7nks3
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,9759243157894736,in,145.138.5.37,vasstdc27m7nks3
5,5,e8291466de91b0eb4e1515143c7f74dexy2yr,3vi4t,mobile,phone7,9759243157894736,ru,91.107.221.27,vasstdc27m7nks3
6,6,eef4c687daf977f64fc1d08675c44444raj3s,kjlzx,mobile,phone2,9759243210526315,th,152.235.155.159,j9nl1xmo6fqhcc6
7,7,ab056855c9ca9d36390feae1fa485883issyg,f5f6k,home goods,phone8,9759243210526315,id,3.210.112.183,hnt6hu93a3z1cpc
8,8,d600dc03b11e7d782e1e4dae091b084a1h5ch,h7jjx,home goods,phone9,9759243210526315,th,103.64.157.225,vasstdc27m7nks3
9,9,a58ace8b671a7531c88814bc86b2a34cf0crb,3zpkj,sporting goods,phone4,9759243210526315,za,123.28.123.226,vasstdc27m7nks3


# Feature Engineering

### Convert Time to pd.datetime

In [12]:
real_time = pd.to_datetime(bids['time'])

In [13]:
real_time.head()

0   1970-04-23 22:54:03.157894736
1   1970-04-23 22:54:03.157894736
2   1970-04-23 22:54:03.157894736
3   1970-04-23 22:54:03.157894736
4   1970-04-23 22:54:03.157894736
Name: time, dtype: datetime64[ns]

In [14]:
bids['time'] = real_time

In [15]:
bids.head()

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,1970-04-23 22:54:03.157894736,us,69.166.231.58,vasstdc27m7nks3
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,1970-04-23 22:54:03.157894736,in,50.201.125.84,jmqlhflrzwuay9c
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,1970-04-23 22:54:03.157894736,py,112.54.208.157,vasstdc27m7nks3
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,1970-04-23 22:54:03.157894736,in,18.99.175.133,vasstdc27m7nks3
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,1970-04-23 22:54:03.157894736,in,145.138.5.37,vasstdc27m7nks3


In [16]:
(bids['time'][50] - bids['time'][0]).microseconds

263158

In [17]:
train_n_test.loc[0]

index                                                  0
bidder_id          91a3c57b13234af24875c56fb7e2b2f4rb56a
payment_account    a3d2de7675556553a5f08e4c88d2c228754av
address            a3d2de7675556553a5f08e4c88d2c228vt0u4
outcome                                                0
Name: 0, dtype: object

### Total bit
- calculate total number of bids placed by one bidder

In [19]:
# Total bit
a = bids[['bidder_id','auction']]

In [20]:
b = a.groupby(['bidder_id']).size().reset_index()

In [21]:
b

,bidder_id,0
0,001068c415025a009fee375a12cff4fcnht8y,1
1,002d229ffb247009810828f648afc2ef593rb,2
2,0030a2dd87ad2733e0873062e4f83954mkj86,1
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,3
4,00486a11dff552c4bd7696265724ff81yeo9v,20
...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,25075
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,22
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1
6612,ffd62646d600b759a985d45918bd6f0431vmz,664


In [22]:
b = b.rename(columns = {0:'total_bid'})

In [23]:
train_n_test = pd.merge(train_n_test, b, on='bidder_id', how='left')

In [24]:
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0
...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0


### Auction Features
- calculate total number of distinct auctions placed by one bidder

In [25]:
# Total auction
a = bids[['bidder_id','auction','merchandise']]
b = a.groupby(['bidder_id','auction']).size().reset_index()

In [26]:
b = b.groupby(['bidder_id']).size().reset_index()

In [27]:
b = b.rename(columns = {0:'total_auction'})

In [28]:
train_n_test = pd.merge(train_n_test, b, on='bidder_id', how='left')

In [29]:
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0
...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0


### Bid Features
- calculate max, median, and mean bids placed per auction by one bidder

In [30]:
# Max bid per auction
a = bids[['bidder_id','auction','merchandise']]
b = a.groupby(['bidder_id','auction']).size().reset_index()

In [31]:
c = b.groupby(['bidder_id']).max().reset_index()

In [32]:
c = c.rename(columns = {0:'max_bid_per_auction'})

In [33]:
c = c.drop(['auction'], axis=1)

In [34]:
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')

In [35]:
# Median bid per auction
c = b.groupby(['bidder_id']).median().reset_index()
c = c.rename(columns = {0:'median_bid_per_auction'})

In [36]:
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')

In [37]:
# Mean bid per auction
c = b.groupby(['bidder_id']).mean().reset_index()
c = c.rename(columns = {0:'mean_bid_per_auction'})

In [38]:
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')

In [39]:
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130
...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000


### Merchandise Features
- calculate max, median, and mean bids placed per merchandise by one bidder

In [40]:
# Total merchandise, Max and Median and Mean bid per merchandise
#a = bids[['bidder_id','merchandise','device']]
#b = a.groupby(['bidder_id','merchandise']).size().reset_index()

In [41]:
#c = b.groupby(['bidder_id']).size().reset_index()
#c = c.rename(columns = {0:'total_merchandise'})

In [42]:
#train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')

In [43]:
#c = c.drop(['merchandise'], axis=1)
#train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')

In [44]:
#c = b.groupby(['bidder_id']).max().reset_index()
#c = c.rename(columns = {0:'max_bid_per_merchandise'})
#train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')

#c = b.groupby(['bidder_id']).median().reset_index()
#c = c.rename(columns = {0:'median_bid_per_merchandise'})
#train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')

#c = b.groupby(['bidder_id']).mean().reset_index()
#c = c.rename(columns = {0:'mean_bid_per_merchandise'})
#train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')

In [45]:
#train_n_test

In [46]:
#train_n_test['total_bid'].describe()

In [47]:
#train_n_test['total_merchandise'].describe()

In [48]:
#Drop Total merchandise, Max and Median and Mean bid per merchandise
#train_n_test = train_n_test.drop(['total_merchandise','merchandise','max_bid_per_merchandise','median_bid_per_merchandise','mean_bid_per_merchandise'], axis=1)

### Device Features
- calculate total devices, most used devices, and rare device used by one bidder

In [50]:
# Total device
a = bids[['bidder_id','merchandise','device']]
b = a.groupby(['bidder_id','device']).size().reset_index()

In [51]:
c = b.groupby(['bidder_id']).size().reset_index()
c = c.rename(columns = {0:'total_device'})
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')

In [52]:
# most used device
c = b.groupby(['bidder_id']).max().reset_index()
c = c.rename(columns = {'device':'most_used_device', 0:'bid_most_used_device'})

In [53]:
c

,bidder_id,most_used_device,bid_most_used_device
0,001068c415025a009fee375a12cff4fcnht8y,phone561,1
1,002d229ffb247009810828f648afc2ef593rb,phone640,1
2,0030a2dd87ad2733e0873062e4f83954mkj86,phone313,1
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,phone420,1
4,00486a11dff552c4bd7696265724ff81yeo9v,phone788,7
...,...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,phone996,2432
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,phone97,4
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,phone248,1
6612,ffd62646d600b759a985d45918bd6f0431vmz,phone98,200


In [54]:
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,total_device,most_used_device,bid_most_used_device
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,14.0,phone79,6.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,2.0,phone293,2.0
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,2.0,phone4,3.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,1.0,phone58,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,53.0,phone95,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,9.0,phone786,421.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,29.0,phone986,22.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,376.0,phone996,349.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,4.0,phone57,2.0


In [55]:
# rare device
c = b.groupby(['bidder_id']).min().reset_index()
c = c.rename(columns = {'device':'rare_device', 0:'bid_rare_device'})
c

,bidder_id,rare_device,bid_rare_device
0,001068c415025a009fee375a12cff4fcnht8y,phone561,1
1,002d229ffb247009810828f648afc2ef593rb,phone219,1
2,0030a2dd87ad2733e0873062e4f83954mkj86,phone313,1
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,phone102,1
4,00486a11dff552c4bd7696265724ff81yeo9v,phone167,1
...,...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,phone0,1
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,phone101,1
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,phone248,1
6612,ffd62646d600b759a985d45918bd6f0431vmz,phone0,1


In [56]:
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,total_device,most_used_device,bid_most_used_device,rare_device,bid_rare_device
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,14.0,phone79,6.0,phone1004,1.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,2.0,phone293,2.0,phone286,1.0
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,2.0,phone4,3.0,phone111,1.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,1.0,phone58,1.0,phone58,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,53.0,phone95,19.0,phone101,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,9.0,phone786,421.0,phone110,3.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,29.0,phone986,22.0,phone0,1.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,376.0,phone996,349.0,phone0,1.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,4.0,phone57,2.0,phone17,1.0


### Times Features

### Times dataframe
- calculate start time and end time of an auction

In [57]:
# Times dataframe
times = bids.groupby('auction').time.min().reset_index()

In [58]:
times = times.rename(columns = {'time':'start_time'})
times

,auction,start_time
0,00270,1970-04-23 06:10:49.894736842
1,008vv,1970-04-23 22:56:09.421052631
2,00cwr,1970-04-23 05:14:01.631578947
3,00do0,1970-04-23 22:55:23.842105263
4,00hjy,1970-04-23 22:56:50.368421052
...,...,...
15046,zznnc,1970-04-23 22:59:55.684210526
15047,zzoy7,1970-04-22 11:33:14.105263157
15048,zztha,1970-04-24 01:09:48.947368421
15049,zzzlx,1970-04-23 22:54:24.894736842


In [59]:
times2 = bids.groupby('auction').time.max().reset_index()
times2 = times2.rename(columns = {'time':'end_time'})
times2

,auction,end_time
0,00270,1970-04-23 09:00:12.894736842
1,008vv,1970-04-23 23:13:17.157894736
2,00cwr,1970-04-23 06:03:56.578947368
3,00do0,1970-04-23 23:04:25.210526315
4,00hjy,1970-04-24 02:38:43.842105263
...,...,...
15046,zznnc,1970-04-24 02:31:27.052631578
15047,zzoy7,1970-04-23 09:00:04.368421052
15048,zztha,1970-04-24 01:41:53.631578947
15049,zzzlx,1970-04-24 02:23:53.842105263


In [60]:
times = pd.merge(times, times2, on='auction', how='left')
times

,auction,start_time,end_time
0,00270,1970-04-23 06:10:49.894736842,1970-04-23 09:00:12.894736842
1,008vv,1970-04-23 22:56:09.421052631,1970-04-23 23:13:17.157894736
2,00cwr,1970-04-23 05:14:01.631578947,1970-04-23 06:03:56.578947368
3,00do0,1970-04-23 22:55:23.842105263,1970-04-23 23:04:25.210526315
4,00hjy,1970-04-23 22:56:50.368421052,1970-04-24 02:38:43.842105263
...,...,...,...
15046,zznnc,1970-04-23 22:59:55.684210526,1970-04-24 02:31:27.052631578
15047,zzoy7,1970-04-22 11:33:14.105263157,1970-04-23 09:00:04.368421052
15048,zztha,1970-04-24 01:09:48.947368421,1970-04-24 01:41:53.631578947
15049,zzzlx,1970-04-23 22:54:24.894736842,1970-04-24 02:23:53.842105263


### Duration of auction
- calculate duration of auction by end time - start time

In [61]:
times['duration'] = times.end_time - times.start_time

In [62]:
times['duration'] = pd.to_timedelta(times['duration'])

In [63]:
times['duration'] = times['duration'].dt.total_seconds()

In [64]:
times

,auction,start_time,end_time,duration
0,00270,1970-04-23 06:10:49.894736842,1970-04-23 09:00:12.894736842,10163.000000
1,008vv,1970-04-23 22:56:09.421052631,1970-04-23 23:13:17.157894736,1027.736842
2,00cwr,1970-04-23 05:14:01.631578947,1970-04-23 06:03:56.578947368,2994.947368
3,00do0,1970-04-23 22:55:23.842105263,1970-04-23 23:04:25.210526315,541.368421
4,00hjy,1970-04-23 22:56:50.368421052,1970-04-24 02:38:43.842105263,13313.473684
...,...,...,...,...
15046,zznnc,1970-04-23 22:59:55.684210526,1970-04-24 02:31:27.052631578,12691.368421
15047,zzoy7,1970-04-22 11:33:14.105263157,1970-04-23 09:00:04.368421052,77210.263158
15048,zztha,1970-04-24 01:09:48.947368421,1970-04-24 01:41:53.631578947,1924.684211
15049,zzzlx,1970-04-23 22:54:24.894736842,1970-04-24 02:23:53.842105263,12568.947368


In [65]:
bids_with_times = pd.merge(bids, times, on='auction', how='left')

In [66]:
bids_with_times

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url,start_time,end_time,duration
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,1970-04-23 22:54:03.157894736,us,69.166.231.58,vasstdc27m7nks3,1970-04-23 22:54:03.157894736,1970-04-24 02:41:14.210526315,13631.052632
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,1970-04-23 22:54:03.157894736,in,50.201.125.84,jmqlhflrzwuay9c,1970-04-23 22:54:03.157894736,1970-04-24 02:41:25.000000000,13641.842105
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,1970-04-23 22:54:03.157894736,py,112.54.208.157,vasstdc27m7nks3,1970-04-23 22:54:03.157894736,1970-04-24 02:41:21.631578947,13638.473684
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,1970-04-23 22:54:03.157894736,in,18.99.175.133,vasstdc27m7nks3,1970-04-23 22:54:03.157894736,1970-04-24 02:41:25.210526315,13642.052632
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,1970-04-23 22:54:03.157894736,in,145.138.5.37,vasstdc27m7nks3,1970-04-23 22:54:03.157894736,1970-04-24 02:41:25.210526315,13642.052632
...,...,...,...,...,...,...,...,...,...,...,...,...
7656329,7656329,626159dd6f2228ede002d9f9340f75b7puk8d,3e64w,jewelry,phone91,1970-04-23 09:00:22.052631578,ru,140.204.227.63,cghhmomsaxi6pug,1970-04-23 08:46:32.473684210,1970-04-23 09:00:22.052631578,829.578947
7656330,7656330,a318ea333ceee1ba39a494476386136a826dv,xn0y0,mobile,phone236,1970-04-23 09:00:22.052631578,pl,24.232.159.118,wgggpdg2gx5pesn,1970-04-22 11:32:24.263157894,1970-04-23 09:00:22.052631578,77277.789474
7656331,7656331,f5b2bbad20d1d7ded3ed960393bec0f40u6hn,gja6c,sporting goods,phone80,1970-04-23 09:00:22.052631578,za,80.237.28.246,5xgysg14grlersa,1970-04-22 11:31:57.157894736,1970-04-23 09:00:22.052631578,77304.894737
7656332,7656332,d4bd412590f5106b9d887a43c51b254eldo4f,hmwk8,jewelry,phone349,1970-04-23 09:00:22.052631578,my,91.162.27.152,bhtrek44bzi2wfl,1970-04-22 11:35:17.894736842,1970-04-23 09:00:22.052631578,77104.157895


### Delta times
- calculate time difference from start time of auction to time that bidder placed first bid on that auction

- calculate time difference from time that bidder placed last bid on auction to end time of that auction

In [67]:
# Delta time since start, until end
bids_with_times['dt_since_start'] = bids_with_times.time - bids_with_times.start_time
bids_with_times['dt_since_start'] = bids_with_times['dt_since_start'].dt.total_seconds()
bids_with_times['dt_until_end'] = bids_with_times.end_time - bids_with_times.time
bids_with_times['dt_until_end'] = bids_with_times['dt_until_end'].dt.total_seconds()

In [68]:
bids_with_times

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url,start_time,end_time,duration,dt_since_start,dt_until_end
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,1970-04-23 22:54:03.157894736,us,69.166.231.58,vasstdc27m7nks3,1970-04-23 22:54:03.157894736,1970-04-24 02:41:14.210526315,13631.052632,0.000000,13631.052632
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,1970-04-23 22:54:03.157894736,in,50.201.125.84,jmqlhflrzwuay9c,1970-04-23 22:54:03.157894736,1970-04-24 02:41:25.000000000,13641.842105,0.000000,13641.842105
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,1970-04-23 22:54:03.157894736,py,112.54.208.157,vasstdc27m7nks3,1970-04-23 22:54:03.157894736,1970-04-24 02:41:21.631578947,13638.473684,0.000000,13638.473684
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,1970-04-23 22:54:03.157894736,in,18.99.175.133,vasstdc27m7nks3,1970-04-23 22:54:03.157894736,1970-04-24 02:41:25.210526315,13642.052632,0.000000,13642.052632
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,1970-04-23 22:54:03.157894736,in,145.138.5.37,vasstdc27m7nks3,1970-04-23 22:54:03.157894736,1970-04-24 02:41:25.210526315,13642.052632,0.000000,13642.052632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7656329,7656329,626159dd6f2228ede002d9f9340f75b7puk8d,3e64w,jewelry,phone91,1970-04-23 09:00:22.052631578,ru,140.204.227.63,cghhmomsaxi6pug,1970-04-23 08:46:32.473684210,1970-04-23 09:00:22.052631578,829.578947,829.578947,0.000000
7656330,7656330,a318ea333ceee1ba39a494476386136a826dv,xn0y0,mobile,phone236,1970-04-23 09:00:22.052631578,pl,24.232.159.118,wgggpdg2gx5pesn,1970-04-22 11:32:24.263157894,1970-04-23 09:00:22.052631578,77277.789474,77277.789474,0.000000
7656331,7656331,f5b2bbad20d1d7ded3ed960393bec0f40u6hn,gja6c,sporting goods,phone80,1970-04-23 09:00:22.052631578,za,80.237.28.246,5xgysg14grlersa,1970-04-22 11:31:57.157894736,1970-04-23 09:00:22.052631578,77304.894737,77304.894737,0.000000
7656332,7656332,d4bd412590f5106b9d887a43c51b254eldo4f,hmwk8,jewelry,phone349,1970-04-23 09:00:22.052631578,my,91.162.27.152,bhtrek44bzi2wfl,1970-04-22 11:35:17.894736842,1970-04-23 09:00:22.052631578,77104.157895,77104.157895,0.000000


### Delta time 2
- calculate median of time difference since start of bidder on all auctions
- calculate median of time difference until end of bidder on all auctions

In [69]:
b = bids_with_times.groupby('bidder_id').dt_since_start.median().reset_index()
b = b.rename(columns = {'dt_since_start':'median_dt_since_start'})

In [70]:
b

,bidder_id,median_dt_since_start
0,001068c415025a009fee375a12cff4fcnht8y,74390.578947
1,002d229ffb247009810828f648afc2ef593rb,145.526316
2,0030a2dd87ad2733e0873062e4f83954mkj86,72636.894737
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,7466.789474
4,00486a11dff552c4bd7696265724ff81yeo9v,9833.605263
...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,5373.736842
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,8155.368421
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,10443.052632
6612,ffd62646d600b759a985d45918bd6f0431vmz,4554.289474


In [71]:
train_n_test = pd.merge(train_n_test, b, on='bidder_id', how='left')

In [72]:
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,total_device,most_used_device,bid_most_used_device,rare_device,bid_rare_device,median_dt_since_start
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,14.0,phone79,6.0,phone1004,1.0,4489.263158
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,2.0,phone293,2.0,phone286,1.0,6793.263158
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,2.0,phone4,3.0,phone111,1.0,7707.289474
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,1.0,phone58,1.0,phone58,1.0,74481.421053
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,53.0,phone95,19.0,phone101,1.0,642.736842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,9.0,phone786,421.0,phone110,3.0,65660.157895
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,29.0,phone986,22.0,phone0,1.0,5046.789474
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,376.0,phone996,349.0,phone0,1.0,8635.315789
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,4.0,phone57,2.0,phone17,1.0,4494.473684


In [73]:
b = bids_with_times.groupby('bidder_id').dt_until_end.median().reset_index()
b = b.rename(columns = {'dt_until_end':'median_dt_until_end'})
b

,bidder_id,median_dt_until_end
0,001068c415025a009fee375a12cff4fcnht8y,624.894737
1,002d229ffb247009810828f648afc2ef593rb,1799.157895
2,0030a2dd87ad2733e0873062e4f83954mkj86,4667.315789
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,8287.263158
4,00486a11dff552c4bd7696265724ff81yeo9v,57412.631579
...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,5899.842105
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,8584.315789
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,66712.947368
6612,ffd62646d600b759a985d45918bd6f0431vmz,4667.736842


In [74]:
train_n_test = pd.merge(train_n_test, b, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,total_device,most_used_device,bid_most_used_device,rare_device,bid_rare_device,median_dt_since_start,median_dt_until_end
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,14.0,phone79,6.0,phone1004,1.0,4489.263158,5956.973684
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,2.0,phone293,2.0,phone286,1.0,6793.263158,6848.789474
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,2.0,phone4,3.0,phone111,1.0,7707.289474,3057.789474
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,1.0,phone58,1.0,phone58,1.0,74481.421053,2774.684211
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,53.0,phone95,19.0,phone101,1.0,642.736842,10131.263158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,9.0,phone786,421.0,phone110,3.0,65660.157895,7231.315789
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,29.0,phone986,22.0,phone0,1.0,5046.789474,6678.578947
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,376.0,phone996,349.0,phone0,1.0,8635.315789,53643.578947
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,4.0,phone57,2.0,phone17,1.0,4494.473684,71239.263158


### Delta time 3
- calculate median and min time difference between two bids by same bidder on an auction

In [75]:
# Median, Min difference time between two bids by same bidder
a = bids_with_times[['bidder_id','auction','time']].reset_index()

In [76]:
b = a.sort_values(['bidder_id','time']).reset_index()

In [77]:
b

,level_0,index,bidder_id,auction,time
0,7179832,7179832,001068c415025a009fee375a12cff4fcnht8y,4ifac,1970-04-23 08:12:25.052631578
1,1281292,1281292,002d229ffb247009810828f648afc2ef593rb,2tdw2,1970-04-24 00:59:04.105263157
2,1281311,1281311,002d229ffb247009810828f648afc2ef593rb,2tdw2,1970-04-24 00:59:04.210526315
3,6805028,6805028,0030a2dd87ad2733e0873062e4f83954mkj86,obbny,1970-04-23 07:42:33.947368421
4,3967330,3967330,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,obbny,1970-04-22 13:46:58.631578947
...,...,...,...,...,...
7656329,2336021,2336021,ffd62646d600b759a985d45918bd6f0431vmz,dwy5m,1970-04-24 02:40:02.947368421
7656330,2337626,2337626,ffd62646d600b759a985d45918bd6f0431vmz,kh2ef,1970-04-24 02:40:11.421052631
7656331,2344645,2344645,ffd62646d600b759a985d45918bd6f0431vmz,wc5b0,1970-04-24 02:40:50.052631578
7656332,2350507,2350507,ffd62646d600b759a985d45918bd6f0431vmz,wc5b0,1970-04-24 02:41:21.631578947


In [78]:
b.loc[0].time

Timestamp('1970-04-23 08:12:25.052631578')

In [79]:
b['bidder_id_prev'] = pd.Series(np.append([np.nan], b.bidder_id.values[0:-1]))

In [80]:
b['time_prev'] = pd.Series(np.append([pd.NaT], b.time.values[0:-1]))

In [81]:
b['time_prev'] = pd.to_datetime(b['time_prev'])

In [82]:
b

,level_0,index,bidder_id,auction,time,bidder_id_prev,time_prev
0,7179832,7179832,001068c415025a009fee375a12cff4fcnht8y,4ifac,1970-04-23 08:12:25.052631578,NaN,NaT
1,1281292,1281292,002d229ffb247009810828f648afc2ef593rb,2tdw2,1970-04-24 00:59:04.105263157,001068c415025a009fee375a12cff4fcnht8y,1970-04-23 08:12:25.052631578
2,1281311,1281311,002d229ffb247009810828f648afc2ef593rb,2tdw2,1970-04-24 00:59:04.210526315,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.105263157
3,6805028,6805028,0030a2dd87ad2733e0873062e4f83954mkj86,obbny,1970-04-23 07:42:33.947368421,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.210526315
4,3967330,3967330,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,obbny,1970-04-22 13:46:58.631578947,0030a2dd87ad2733e0873062e4f83954mkj86,1970-04-23 07:42:33.947368421
...,...,...,...,...,...,...,...
7656329,2336021,2336021,ffd62646d600b759a985d45918bd6f0431vmz,dwy5m,1970-04-24 02:40:02.947368421,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:39:57.315789473
7656330,2337626,2337626,ffd62646d600b759a985d45918bd6f0431vmz,kh2ef,1970-04-24 02:40:11.421052631,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:02.947368421
7656331,2344645,2344645,ffd62646d600b759a985d45918bd6f0431vmz,wc5b0,1970-04-24 02:40:50.052631578,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:11.421052631
7656332,2350507,2350507,ffd62646d600b759a985d45918bd6f0431vmz,wc5b0,1970-04-24 02:41:21.631578947,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:50.052631578


In [83]:
b['dt_time'] = b['time'] - b['time_prev']
b['dt_time'] = b['dt_time'].dt.total_seconds()
b

,level_0,index,bidder_id,auction,time,bidder_id_prev,time_prev,dt_time
0,7179832,7179832,001068c415025a009fee375a12cff4fcnht8y,4ifac,1970-04-23 08:12:25.052631578,NaN,NaT,NaN
1,1281292,1281292,002d229ffb247009810828f648afc2ef593rb,2tdw2,1970-04-24 00:59:04.105263157,001068c415025a009fee375a12cff4fcnht8y,1970-04-23 08:12:25.052631578,60399.052632
2,1281311,1281311,002d229ffb247009810828f648afc2ef593rb,2tdw2,1970-04-24 00:59:04.210526315,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.105263157,0.105263
3,6805028,6805028,0030a2dd87ad2733e0873062e4f83954mkj86,obbny,1970-04-23 07:42:33.947368421,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.210526315,-62190.263158
4,3967330,3967330,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,obbny,1970-04-22 13:46:58.631578947,0030a2dd87ad2733e0873062e4f83954mkj86,1970-04-23 07:42:33.947368421,-64535.315789
...,...,...,...,...,...,...,...,...
7656329,2336021,2336021,ffd62646d600b759a985d45918bd6f0431vmz,dwy5m,1970-04-24 02:40:02.947368421,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:39:57.315789473,5.631579
7656330,2337626,2337626,ffd62646d600b759a985d45918bd6f0431vmz,kh2ef,1970-04-24 02:40:11.421052631,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:02.947368421,8.473684
7656331,2344645,2344645,ffd62646d600b759a985d45918bd6f0431vmz,wc5b0,1970-04-24 02:40:50.052631578,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:11.421052631,38.631579
7656332,2350507,2350507,ffd62646d600b759a985d45918bd6f0431vmz,wc5b0,1970-04-24 02:41:21.631578947,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:50.052631578,31.578947


In [84]:
b = b.drop(['level_0', 'index'], axis=1)
c = b[b.bidder_id == b.bidder_id_prev][b.dt_time > 0.0]
c = c.reset_index()
c

D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,index,bidder_id,auction,time,bidder_id_prev,time_prev,dt_time
0,2,002d229ffb247009810828f648afc2ef593rb,2tdw2,1970-04-24 00:59:04.210526315,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.105263157,0.105263
1,5,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,cqsh6,1970-04-23 06:36:45.052631578,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1970-04-22 13:46:58.631578947,60586.421053
2,6,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,efh5o,1970-04-23 08:06:14.315789473,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1970-04-23 06:36:45.052631578,5369.263158
3,8,00486a11dff552c4bd7696265724ff81yeo9v,6plix,1970-04-22 11:44:01.157894736,00486a11dff552c4bd7696265724ff81yeo9v,1970-04-22 11:43:56.526315789,4.631579
4,9,00486a11dff552c4bd7696265724ff81yeo9v,gst86,1970-04-22 11:55:39.684210526,00486a11dff552c4bd7696265724ff81yeo9v,1970-04-22 11:44:01.157894736,698.526316
...,...,...,...,...,...,...,...
6836899,7656328,ffd62646d600b759a985d45918bd6f0431vmz,b779a,1970-04-24 02:39:57.315789473,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:39:57.105263157,0.210526
6836900,7656329,ffd62646d600b759a985d45918bd6f0431vmz,dwy5m,1970-04-24 02:40:02.947368421,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:39:57.315789473,5.631579
6836901,7656330,ffd62646d600b759a985d45918bd6f0431vmz,kh2ef,1970-04-24 02:40:11.421052631,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:02.947368421,8.473684
6836902,7656331,ffd62646d600b759a985d45918bd6f0431vmz,wc5b0,1970-04-24 02:40:50.052631578,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:11.421052631,38.631579


In [85]:
d = c.groupby('bidder_id').dt_time.median().reset_index()
d = d.rename(columns = {'dt_time': 'median_dt_time_self'})
d

,bidder_id,median_dt_time_self
0,002d229ffb247009810828f648afc2ef593rb,0.105263
1,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,32977.842105
2,00486a11dff552c4bd7696265724ff81yeo9v,917.736842
3,0051aef3fdeacdadba664b9b3b07e04e4coc6,2.736842
4,0053b78cde37c4384a20d2da9aa4272aym4pb,0.421053
...,...,...
5551,ffacbed056cbfaa60c1fcf51f0d381bddr3ly,1948.684211
5552,ffaf0a972a6dcb3910fd6b16045781e2ava5y,4388.157895
5553,ffbc0fdfbf19a8a9116b68714138f2902cc13,0.368421
5554,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,690.368421


In [86]:
train_n_test = pd.merge(train_n_test, d, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,total_device,most_used_device,bid_most_used_device,rare_device,bid_rare_device,median_dt_since_start,median_dt_until_end,median_dt_time_self
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,14.0,phone79,6.0,phone1004,1.0,4489.263158,5956.973684,345.842105
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,2.0,phone293,2.0,phone286,1.0,6793.263158,6848.789474,3233.578947
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,2.0,phone4,3.0,phone111,1.0,7707.289474,3057.789474,2532.052632
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,1.0,phone58,1.0,phone58,1.0,74481.421053,2774.684211,NaN
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,53.0,phone95,19.0,phone101,1.0,642.736842,10131.263158,13.684211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,9.0,phone786,421.0,phone110,3.0,65660.157895,7231.315789,1.421053
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,29.0,phone986,22.0,phone0,1.0,5046.789474,6678.578947,95.842105
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,376.0,phone996,349.0,phone0,1.0,8635.315789,53643.578947,6.578947
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,4.0,phone57,2.0,phone17,1.0,4494.473684,71239.263158,1519.736842


In [87]:
d = c.groupby('bidder_id').dt_time.min().reset_index()
d = d.rename(columns = {'dt_time': 'min_dt_time_self'})
d

,bidder_id,min_dt_time_self
0,002d229ffb247009810828f648afc2ef593rb,0.105263
1,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,5369.263158
2,00486a11dff552c4bd7696265724ff81yeo9v,0.368421
3,0051aef3fdeacdadba664b9b3b07e04e4coc6,0.052632
4,0053b78cde37c4384a20d2da9aa4272aym4pb,0.052632
...,...,...
5551,ffacbed056cbfaa60c1fcf51f0d381bddr3ly,2.736842
5552,ffaf0a972a6dcb3910fd6b16045781e2ava5y,3448.473684
5553,ffbc0fdfbf19a8a9116b68714138f2902cc13,0.052632
5554,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,1.368421


In [88]:
train_n_test = pd.merge(train_n_test, d, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,total_device,most_used_device,bid_most_used_device,rare_device,bid_rare_device,median_dt_since_start,median_dt_until_end,median_dt_time_self,min_dt_time_self
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,14.0,phone79,6.0,phone1004,1.0,4489.263158,5956.973684,345.842105,5.210526
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,2.0,phone293,2.0,phone286,1.0,6793.263158,6848.789474,3233.578947,1989.315789
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,2.0,phone4,3.0,phone111,1.0,7707.289474,3057.789474,2532.052632,1450.842105
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,1.0,phone58,1.0,phone58,1.0,74481.421053,2774.684211,NaN,NaN
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,53.0,phone95,19.0,phone101,1.0,642.736842,10131.263158,13.684211,0.052632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,9.0,phone786,421.0,phone110,3.0,65660.157895,7231.315789,1.421053,0.052632
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,29.0,phone986,22.0,phone0,1.0,5046.789474,6678.578947,95.842105,0.473684
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,376.0,phone996,349.0,phone0,1.0,8635.315789,53643.578947,6.578947,0.052632
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,4.0,phone57,2.0,phone17,1.0,4494.473684,71239.263158,1519.736842,459.684211


### Delta time 4
- calculate median and min time difference between two bids by bidder and the previous other bidder on same auction

In [89]:
# Median, Min difference time between two bids by two bidders in same auction
a = bids_with_times[['bidder_id','auction','time']].reset_index()
b = a.sort_values(['auction','time']).reset_index()
b

,level_0,index,bidder_id,auction,time
0,5826422,5826422,92af1e40713e077ef87f5352fb56772fjnzm7,00270,1970-04-23 06:10:49.894736842
1,5834284,5834284,a939f51234ad2c44eb9ccc84e754f41foiv7g,00270,1970-04-23 06:11:38.210526315
2,5850448,5850448,92af1e40713e077ef87f5352fb56772fjnzm7,00270,1970-04-23 06:13:15.789473684
3,5884054,5884054,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,1970-04-23 06:16:32.473684210
4,5898318,5898318,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,1970-04-23 06:17:54.473684210
...,...,...,...,...,...
7656329,7646305,7646305,3e2f6796d57de4ed3e311de6832755938jue0,zzzy7,1970-04-23 08:59:36.894736842
7656330,7648070,7648070,a93529c3244a9e47aa1527e0fa641e1225h0t,zzzy7,1970-04-23 08:59:45.052631578
7656331,7648153,7648153,414af5edf09c473bde75644b76a68e7feesrw,zzzy7,1970-04-23 08:59:45.473684210
7656332,7649363,7649363,0ea62aaa9c3ffcc6db584cb69c1f6c4bcripp,zzzy7,1970-04-23 08:59:51.157894736


In [90]:
b = b.drop(['level_0', 'index'], axis=1)

In [91]:
b['bidder_id_prev'] = pd.Series(np.append([np.nan], b.bidder_id.values[0:-1]))
b['time_prev'] = pd.Series(np.append([pd.NaT], b.time.values[0:-1]))
b['time_prev'] = pd.to_datetime(b['time_prev'])
b['auction_prev'] = pd.Series(np.append([np.nan], b.auction.values[0:-1]))
b

,bidder_id,auction,time,bidder_id_prev,time_prev,auction_prev
0,92af1e40713e077ef87f5352fb56772fjnzm7,00270,1970-04-23 06:10:49.894736842,NaN,NaT,NaN
1,a939f51234ad2c44eb9ccc84e754f41foiv7g,00270,1970-04-23 06:11:38.210526315,92af1e40713e077ef87f5352fb56772fjnzm7,1970-04-23 06:10:49.894736842,00270
2,92af1e40713e077ef87f5352fb56772fjnzm7,00270,1970-04-23 06:13:15.789473684,a939f51234ad2c44eb9ccc84e754f41foiv7g,1970-04-23 06:11:38.210526315,00270
3,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,1970-04-23 06:16:32.473684210,92af1e40713e077ef87f5352fb56772fjnzm7,1970-04-23 06:13:15.789473684,00270
4,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,1970-04-23 06:17:54.473684210,9655ccc7c0c193f1549475f02c54dce45kjw7,1970-04-23 06:16:32.473684210,00270
...,...,...,...,...,...,...
7656329,3e2f6796d57de4ed3e311de6832755938jue0,zzzy7,1970-04-23 08:59:36.894736842,197e90906939267ace2422e78e8e83889znqp,1970-04-23 08:59:34.368421052,zzzy7
7656330,a93529c3244a9e47aa1527e0fa641e1225h0t,zzzy7,1970-04-23 08:59:45.052631578,3e2f6796d57de4ed3e311de6832755938jue0,1970-04-23 08:59:36.894736842,zzzy7
7656331,414af5edf09c473bde75644b76a68e7feesrw,zzzy7,1970-04-23 08:59:45.473684210,a93529c3244a9e47aa1527e0fa641e1225h0t,1970-04-23 08:59:45.052631578,zzzy7
7656332,0ea62aaa9c3ffcc6db584cb69c1f6c4bcripp,zzzy7,1970-04-23 08:59:51.157894736,414af5edf09c473bde75644b76a68e7feesrw,1970-04-23 08:59:45.473684210,zzzy7


In [92]:
b['dt_time'] = b['time'] - b['time_prev']
b['dt_time'] = b['dt_time'].dt.total_seconds()
b

,bidder_id,auction,time,bidder_id_prev,time_prev,auction_prev,dt_time
0,92af1e40713e077ef87f5352fb56772fjnzm7,00270,1970-04-23 06:10:49.894736842,NaN,NaT,NaN,NaN
1,a939f51234ad2c44eb9ccc84e754f41foiv7g,00270,1970-04-23 06:11:38.210526315,92af1e40713e077ef87f5352fb56772fjnzm7,1970-04-23 06:10:49.894736842,00270,48.315789
2,92af1e40713e077ef87f5352fb56772fjnzm7,00270,1970-04-23 06:13:15.789473684,a939f51234ad2c44eb9ccc84e754f41foiv7g,1970-04-23 06:11:38.210526315,00270,97.578947
3,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,1970-04-23 06:16:32.473684210,92af1e40713e077ef87f5352fb56772fjnzm7,1970-04-23 06:13:15.789473684,00270,196.684211
4,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,1970-04-23 06:17:54.473684210,9655ccc7c0c193f1549475f02c54dce45kjw7,1970-04-23 06:16:32.473684210,00270,82.000000
...,...,...,...,...,...,...,...
7656329,3e2f6796d57de4ed3e311de6832755938jue0,zzzy7,1970-04-23 08:59:36.894736842,197e90906939267ace2422e78e8e83889znqp,1970-04-23 08:59:34.368421052,zzzy7,2.526316
7656330,a93529c3244a9e47aa1527e0fa641e1225h0t,zzzy7,1970-04-23 08:59:45.052631578,3e2f6796d57de4ed3e311de6832755938jue0,1970-04-23 08:59:36.894736842,zzzy7,8.157895
7656331,414af5edf09c473bde75644b76a68e7feesrw,zzzy7,1970-04-23 08:59:45.473684210,a93529c3244a9e47aa1527e0fa641e1225h0t,1970-04-23 08:59:45.052631578,zzzy7,0.421053
7656332,0ea62aaa9c3ffcc6db584cb69c1f6c4bcripp,zzzy7,1970-04-23 08:59:51.157894736,414af5edf09c473bde75644b76a68e7feesrw,1970-04-23 08:59:45.473684210,zzzy7,5.684211


In [93]:
c = b[b.bidder_id != b.bidder_id_prev][b.auction == b.auction_prev]
c = c.reset_index()
c

D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,index,bidder_id,auction,time,bidder_id_prev,time_prev,auction_prev,dt_time
0,1,a939f51234ad2c44eb9ccc84e754f41foiv7g,00270,1970-04-23 06:11:38.210526315,92af1e40713e077ef87f5352fb56772fjnzm7,1970-04-23 06:10:49.894736842,00270,48.315789
1,2,92af1e40713e077ef87f5352fb56772fjnzm7,00270,1970-04-23 06:13:15.789473684,a939f51234ad2c44eb9ccc84e754f41foiv7g,1970-04-23 06:11:38.210526315,00270,97.578947
2,3,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,1970-04-23 06:16:32.473684210,92af1e40713e077ef87f5352fb56772fjnzm7,1970-04-23 06:13:15.789473684,00270,196.684211
3,6,84c4b58a1491da3a30710bcdf61f87054xfq6,00270,1970-04-23 06:19:10.684210526,9655ccc7c0c193f1549475f02c54dce45kjw7,1970-04-23 06:19:02.947368421,00270,7.736842
4,7,92af1e40713e077ef87f5352fb56772fjnzm7,00270,1970-04-23 06:19:31.842105263,84c4b58a1491da3a30710bcdf61f87054xfq6,1970-04-23 06:19:10.684210526,00270,21.157895
...,...,...,...,...,...,...,...,...
5566949,7656329,3e2f6796d57de4ed3e311de6832755938jue0,zzzy7,1970-04-23 08:59:36.894736842,197e90906939267ace2422e78e8e83889znqp,1970-04-23 08:59:34.368421052,zzzy7,2.526316
5566950,7656330,a93529c3244a9e47aa1527e0fa641e1225h0t,zzzy7,1970-04-23 08:59:45.052631578,3e2f6796d57de4ed3e311de6832755938jue0,1970-04-23 08:59:36.894736842,zzzy7,8.157895
5566951,7656331,414af5edf09c473bde75644b76a68e7feesrw,zzzy7,1970-04-23 08:59:45.473684210,a93529c3244a9e47aa1527e0fa641e1225h0t,1970-04-23 08:59:45.052631578,zzzy7,0.421053
5566952,7656332,0ea62aaa9c3ffcc6db584cb69c1f6c4bcripp,zzzy7,1970-04-23 08:59:51.157894736,414af5edf09c473bde75644b76a68e7feesrw,1970-04-23 08:59:45.473684210,zzzy7,5.684211


In [94]:
d = c.groupby('bidder_id').dt_time.median().reset_index()
d = d.rename(columns = {'dt_time': 'median_dt_time_other'})
d

,bidder_id,median_dt_time_other
0,001068c415025a009fee375a12cff4fcnht8y,3.842105
1,002d229ffb247009810828f648afc2ef593rb,0.052632
2,0030a2dd87ad2733e0873062e4f83954mkj86,0.736842
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1.000000
4,00486a11dff552c4bd7696265724ff81yeo9v,1.210526
...,...,...
6605,ffbc0fdfbf19a8a9116b68714138f2902cc13,0.736842
6606,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,0.394737
6607,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,7.315789
6608,ffd62646d600b759a985d45918bd6f0431vmz,7.263158


In [95]:
train_n_test = pd.merge(train_n_test, d, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,total_device,most_used_device,bid_most_used_device,rare_device,bid_rare_device,median_dt_since_start,median_dt_until_end,median_dt_time_self,min_dt_time_self,median_dt_time_other
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,14.0,phone79,6.0,phone1004,1.0,4489.263158,5956.973684,345.842105,5.210526,8.105263
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,2.0,phone293,2.0,phone286,1.0,6793.263158,6848.789474,3233.578947,1989.315789,0.052632
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,2.0,phone4,3.0,phone111,1.0,7707.289474,3057.789474,2532.052632,1450.842105,12.236842
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,1.0,phone58,1.0,phone58,1.0,74481.421053,2774.684211,NaN,NaN,17.368421
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,53.0,phone95,19.0,phone101,1.0,642.736842,10131.263158,13.684211,0.052632,0.315789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,9.0,phone786,421.0,phone110,3.0,65660.157895,7231.315789,1.421053,0.052632,0.736842
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,29.0,phone986,22.0,phone0,1.0,5046.789474,6678.578947,95.842105,0.473684,2.000000
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,376.0,phone996,349.0,phone0,1.0,8635.315789,53643.578947,6.578947,0.052632,3.526316
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,4.0,phone57,2.0,phone17,1.0,4494.473684,71239.263158,1519.736842,459.684211,1.000000


In [96]:
d = c.groupby('bidder_id').dt_time.min().reset_index()
d = d.rename(columns = {'dt_time': 'min_dt_time_others'})
d

,bidder_id,min_dt_time_others
0,001068c415025a009fee375a12cff4fcnht8y,3.842105
1,002d229ffb247009810828f648afc2ef593rb,0.052632
2,0030a2dd87ad2733e0873062e4f83954mkj86,0.736842
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,0.526316
4,00486a11dff552c4bd7696265724ff81yeo9v,0.000000
...,...,...
6605,ffbc0fdfbf19a8a9116b68714138f2902cc13,0.000000
6606,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,0.000000
6607,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,7.315789
6608,ffd62646d600b759a985d45918bd6f0431vmz,0.000000


In [97]:
train_n_test = pd.merge(train_n_test, d, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,most_used_device,bid_most_used_device,rare_device,bid_rare_device,median_dt_since_start,median_dt_until_end,median_dt_time_self,min_dt_time_self,median_dt_time_other,min_dt_time_others
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,phone79,6.0,phone1004,1.0,4489.263158,5956.973684,345.842105,5.210526,8.105263,0.000000
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,phone293,2.0,phone286,1.0,6793.263158,6848.789474,3233.578947,1989.315789,0.052632,0.000000
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,phone4,3.0,phone111,1.0,7707.289474,3057.789474,2532.052632,1450.842105,12.236842,4.157895
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,phone58,1.0,phone58,1.0,74481.421053,2774.684211,NaN,NaN,17.368421,17.368421
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,phone95,19.0,phone101,1.0,642.736842,10131.263158,13.684211,0.052632,0.315789,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,phone786,421.0,phone110,3.0,65660.157895,7231.315789,1.421053,0.052632,0.736842,0.000000
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,phone986,22.0,phone0,1.0,5046.789474,6678.578947,95.842105,0.473684,2.000000,0.000000
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,phone996,349.0,phone0,1.0,8635.315789,53643.578947,6.578947,0.052632,3.526316,0.000000
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,phone57,2.0,phone17,1.0,4494.473684,71239.263158,1519.736842,459.684211,1.000000,0.526316


### Country Features
- calculate total countries, most frequent country, rare country used by bidder
- calculate median country used per auction

In [98]:
# Total country, most frequent country, rare country, median country per auction
a = bids[['bidder_id','auction','country']]
b = a.groupby(['bidder_id','country']).size().reset_index()
b

,bidder_id,country,0
0,001068c415025a009fee375a12cff4fcnht8y,bn,1
1,002d229ffb247009810828f648afc2ef593rb,sg,2
2,0030a2dd87ad2733e0873062e4f83954mkj86,ir,1
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,id,3
4,00486a11dff552c4bd7696265724ff81yeo9v,ng,20
...,...,...,...
84153,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,ng,12
84154,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,us,1
84155,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,es,1
84156,ffd62646d600b759a985d45918bd6f0431vmz,id,664


In [99]:
b = b.groupby(['bidder_id']).size().reset_index()
b

,bidder_id,0
0,001068c415025a009fee375a12cff4fcnht8y,1
1,002d229ffb247009810828f648afc2ef593rb,1
2,0030a2dd87ad2733e0873062e4f83954mkj86,1
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1
4,00486a11dff552c4bd7696265724ff81yeo9v,1
...,...,...
6604,ffbc0fdfbf19a8a9116b68714138f2902cc13,102
6605,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,6
6606,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1
6607,ffd62646d600b759a985d45918bd6f0431vmz,1


In [100]:
b = b.rename(columns = {0:'total_country'})
train_n_test = pd.merge(train_n_test, b, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,bid_most_used_device,rare_device,bid_rare_device,median_dt_since_start,median_dt_until_end,median_dt_time_self,min_dt_time_self,median_dt_time_other,min_dt_time_others,total_country
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,6.0,phone1004,1.0,4489.263158,5956.973684,345.842105,5.210526,8.105263,0.000000,6.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,2.0,phone286,1.0,6793.263158,6848.789474,3233.578947,1989.315789,0.052632,0.000000,1.0
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,3.0,phone111,1.0,7707.289474,3057.789474,2532.052632,1450.842105,12.236842,4.157895,1.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,phone58,1.0,74481.421053,2774.684211,NaN,NaN,17.368421,17.368421,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,19.0,phone101,1.0,642.736842,10131.263158,13.684211,0.052632,0.315789,0.000000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,421.0,phone110,3.0,65660.157895,7231.315789,1.421053,0.052632,0.736842,0.000000,5.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,22.0,phone0,1.0,5046.789474,6678.578947,95.842105,0.473684,2.000000,0.000000,10.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,349.0,phone0,1.0,8635.315789,53643.578947,6.578947,0.052632,3.526316,0.000000,86.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,2.0,phone17,1.0,4494.473684,71239.263158,1519.736842,459.684211,1.000000,0.526316,1.0


In [101]:
b = a.groupby(['bidder_id','country']).size().reset_index()
c = b.groupby(['bidder_id']).max().reset_index()
c = c.rename(columns = {'country':'most_freq_country', 0:'amount_most_freq_country'})

In [102]:
c

,bidder_id,most_freq_country,amount_most_freq_country
0,001068c415025a009fee375a12cff4fcnht8y,bn,1
1,002d229ffb247009810828f648afc2ef593rb,sg,2
2,0030a2dd87ad2733e0873062e4f83954mkj86,ir,1
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,id,3
4,00486a11dff552c4bd7696265724ff81yeo9v,ng,20
...,...,...,...
6604,ffbc0fdfbf19a8a9116b68714138f2902cc13,zw,18576
6605,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,us,12
6606,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,es,1
6607,ffd62646d600b759a985d45918bd6f0431vmz,id,664


In [103]:
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,bid_rare_device,median_dt_since_start,median_dt_until_end,median_dt_time_self,min_dt_time_self,median_dt_time_other,min_dt_time_others,total_country,most_freq_country,amount_most_freq_country
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,1.0,4489.263158,5956.973684,345.842105,5.210526,8.105263,0.000000,6.0,zm,13.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,1.0,6793.263158,6848.789474,3233.578947,1989.315789,0.052632,0.000000,1.0,id,3.0
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,1.0,7707.289474,3057.789474,2532.052632,1450.842105,12.236842,4.157895,1.0,id,4.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,74481.421053,2774.684211,NaN,NaN,17.368421,17.368421,1.0,my,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,1.0,642.736842,10131.263158,13.684211,0.052632,0.315789,0.000000,2.0,us,154.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,3.0,65660.157895,7231.315789,1.421053,0.052632,0.736842,0.000000,5.0,us,645.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,1.0,5046.789474,6678.578947,95.842105,0.473684,2.000000,0.000000,10.0,us,45.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,1.0,8635.315789,53643.578947,6.578947,0.052632,3.526316,0.000000,86.0,zw,809.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,1.0,4494.473684,71239.263158,1519.736842,459.684211,1.000000,0.526316,1.0,in,5.0


In [104]:
c = b.groupby(['bidder_id']).min().reset_index()
c = c.rename(columns = {'country':'rare_country', 0:'amount_rare_country'})
c

,bidder_id,rare_country,amount_rare_country
0,001068c415025a009fee375a12cff4fcnht8y,bn,1
1,002d229ffb247009810828f648afc2ef593rb,sg,2
2,0030a2dd87ad2733e0873062e4f83954mkj86,ir,1
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,id,3
4,00486a11dff552c4bd7696265724ff81yeo9v,ng,20
...,...,...,...
6604,ffbc0fdfbf19a8a9116b68714138f2902cc13,ae,1
6605,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,bd,1
6606,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,es,1
6607,ffd62646d600b759a985d45918bd6f0431vmz,id,664


In [105]:
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,median_dt_until_end,median_dt_time_self,min_dt_time_self,median_dt_time_other,min_dt_time_others,total_country,most_freq_country,amount_most_freq_country,rare_country,amount_rare_country
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,5956.973684,345.842105,5.210526,8.105263,0.000000,6.0,zm,13.0,et,1.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,6848.789474,3233.578947,1989.315789,0.052632,0.000000,1.0,id,3.0,id,3.0
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,3057.789474,2532.052632,1450.842105,12.236842,4.157895,1.0,id,4.0,id,4.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,2774.684211,NaN,NaN,17.368421,17.368421,1.0,my,1.0,my,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,10131.263158,13.684211,0.052632,0.315789,0.000000,2.0,us,154.0,in,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,7231.315789,1.421053,0.052632,0.736842,0.000000,5.0,us,645.0,id,3.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,6678.578947,95.842105,0.473684,2.000000,0.000000,10.0,us,45.0,ca,1.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,53643.578947,6.578947,0.052632,3.526316,0.000000,86.0,zw,809.0,al,1.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,71239.263158,1519.736842,459.684211,1.000000,0.526316,1.0,in,5.0,in,5.0


In [106]:
b = a.groupby(['bidder_id','auction','country']).size().reset_index()
b

,bidder_id,auction,country,0
0,001068c415025a009fee375a12cff4fcnht8y,4ifac,bn,1
1,002d229ffb247009810828f648afc2ef593rb,2tdw2,sg,2
2,0030a2dd87ad2733e0873062e4f83954mkj86,obbny,ir,1
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,cqsh6,id,1
4,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,efh5o,id,1
...,...,...,...,...
496566,ffd62646d600b759a985d45918bd6f0431vmz,wthc6,id,1
496567,ffd62646d600b759a985d45918bd6f0431vmz,yfjur,id,7
496568,ffd62646d600b759a985d45918bd6f0431vmz,yv5hw,id,5
496569,ffd62646d600b759a985d45918bd6f0431vmz,zz4dz,id,5


In [107]:
c = b.groupby(['bidder_id','auction']).size().reset_index()
c = c.groupby(['bidder_id']).median().reset_index()
c = c.rename(columns = {0:'median_country_per_auction'})
c

,bidder_id,median_country_per_auction
0,001068c415025a009fee375a12cff4fcnht8y,1.0
1,002d229ffb247009810828f648afc2ef593rb,1.0
2,0030a2dd87ad2733e0873062e4f83954mkj86,1.0
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1.0
4,00486a11dff552c4bd7696265724ff81yeo9v,1.0
...,...,...
6604,ffbc0fdfbf19a8a9116b68714138f2902cc13,1.0
6605,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,1.0
6606,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1.0
6607,ffd62646d600b759a985d45918bd6f0431vmz,1.0


In [108]:
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,median_dt_time_self,min_dt_time_self,median_dt_time_other,min_dt_time_others,total_country,most_freq_country,amount_most_freq_country,rare_country,amount_rare_country,median_country_per_auction
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,345.842105,5.210526,8.105263,0.000000,6.0,zm,13.0,et,1.0,1.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,3233.578947,1989.315789,0.052632,0.000000,1.0,id,3.0,id,3.0,1.0
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,2532.052632,1450.842105,12.236842,4.157895,1.0,id,4.0,id,4.0,1.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,NaN,NaN,17.368421,17.368421,1.0,my,1.0,my,1.0,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,13.684211,0.052632,0.315789,0.000000,2.0,us,154.0,in,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,1.421053,0.052632,0.736842,0.000000,5.0,us,645.0,id,3.0,1.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,95.842105,0.473684,2.000000,0.000000,10.0,us,45.0,ca,1.0,1.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,6.578947,0.052632,3.526316,0.000000,86.0,zw,809.0,al,1.0,1.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,1519.736842,459.684211,1.000000,0.526316,1.0,in,5.0,in,5.0,1.0


In [109]:
c = b.groupby(['bidder_id','auction']).size().reset_index()
c = c.groupby(['bidder_id']).mean().reset_index()
c = c.rename(columns = {0:'mean_country_per_auction'})
c

,bidder_id,mean_country_per_auction
0,001068c415025a009fee375a12cff4fcnht8y,1.000000
1,002d229ffb247009810828f648afc2ef593rb,1.000000
2,0030a2dd87ad2733e0873062e4f83954mkj86,1.000000
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1.000000
4,00486a11dff552c4bd7696265724ff81yeo9v,1.000000
...,...,...
6604,ffbc0fdfbf19a8a9116b68714138f2902cc13,1.180534
6605,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,1.066667
6606,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1.000000
6607,ffd62646d600b759a985d45918bd6f0431vmz,1.000000


In [110]:
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,min_dt_time_self,median_dt_time_other,min_dt_time_others,total_country,most_freq_country,amount_most_freq_country,rare_country,amount_rare_country,median_country_per_auction,mean_country_per_auction
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,5.210526,8.105263,0.000000,6.0,zm,13.0,et,1.0,1.0,1.166667
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,1989.315789,0.052632,0.000000,1.0,id,3.0,id,3.0,1.0,1.000000
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,1450.842105,12.236842,4.157895,1.0,id,4.0,id,4.0,1.0,1.000000
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,NaN,17.368421,17.368421,1.0,my,1.0,my,1.0,1.0,1.000000
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,0.052632,0.315789,0.000000,2.0,us,154.0,in,1.0,1.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,0.052632,0.736842,0.000000,5.0,us,645.0,id,3.0,1.0,1.121951
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,0.473684,2.000000,0.000000,10.0,us,45.0,ca,1.0,1.0,1.093750
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,0.052632,3.526316,0.000000,86.0,zw,809.0,al,1.0,1.0,1.651551
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,459.684211,1.000000,0.526316,1.0,in,5.0,in,5.0,1.0,1.000000


### IP Features
- calculate total ip, most used ip, rare ip used by bidder
- calculate median and mean number of ips per auction
- calculate median and mean number of bids placed per ip

In [111]:
# Total ip, most used ip, rare ip, median and mean ip per auction, median and mean bid per ip
a = bids[['bidder_id','auction','ip']]
b = a.groupby(['bidder_id','ip']).size().reset_index()
b

,bidder_id,ip,0
0,001068c415025a009fee375a12cff4fcnht8y,139.226.147.115,1
1,002d229ffb247009810828f648afc2ef593rb,37.40.254.131,2
2,0030a2dd87ad2733e0873062e4f83954mkj86,21.67.17.162,1
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,115.47.140.180,1
4,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,190.88.89.83,1
...,...,...,...
3598648,ffd62646d600b759a985d45918bd6f0431vmz,7.199.77.86,26
3598649,ffd62646d600b759a985d45918bd6f0431vmz,81.2.190.0,1
3598650,ffd62646d600b759a985d45918bd6f0431vmz,93.227.210.179,27
3598651,ffd62646d600b759a985d45918bd6f0431vmz,93.82.160.29,14


In [112]:
c = b.groupby(['bidder_id']).size().reset_index()
c = c.rename(columns = {0:'total_ip'})
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,median_dt_time_other,min_dt_time_others,total_country,most_freq_country,amount_most_freq_country,rare_country,amount_rare_country,median_country_per_auction,mean_country_per_auction,total_ip
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,8.105263,0.000000,6.0,zm,13.0,et,1.0,1.0,1.166667,20.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,0.052632,0.000000,1.0,id,3.0,id,3.0,1.0,1.000000,3.0
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,12.236842,4.157895,1.0,id,4.0,id,4.0,1.0,1.000000,4.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,17.368421,17.368421,1.0,my,1.0,my,1.0,1.0,1.000000,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,0.315789,0.000000,2.0,us,154.0,in,1.0,1.0,1.000000,123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,0.736842,0.000000,5.0,us,645.0,id,3.0,1.0,1.121951,22.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,2.000000,0.000000,10.0,us,45.0,ca,1.0,1.0,1.093750,49.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,3.526316,0.000000,86.0,zw,809.0,al,1.0,1.0,1.651551,1460.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,1.000000,0.526316,1.0,in,5.0,in,5.0,1.0,1.000000,5.0


In [113]:
b = a.groupby(['bidder_id','ip']).size().reset_index()
c = b.groupby(['bidder_id']).max().reset_index()
c = c.rename(columns = {'ip':'most_used_ip', 0:'amount_most_used_ip'})
c

,bidder_id,most_used_ip,amount_most_used_ip
0,001068c415025a009fee375a12cff4fcnht8y,139.226.147.115,1
1,002d229ffb247009810828f648afc2ef593rb,37.40.254.131,2
2,0030a2dd87ad2733e0873062e4f83954mkj86,21.67.17.162,1
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,44.241.8.179,1
4,00486a11dff552c4bd7696265724ff81yeo9v,90.89.202.121,6
...,...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,99.95.135.200,159
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,90.170.94.43,2
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,67.61.214.36,1
6612,ffd62646d600b759a985d45918bd6f0431vmz,93.82.160.29,37


In [114]:
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,total_country,most_freq_country,amount_most_freq_country,rare_country,amount_rare_country,median_country_per_auction,mean_country_per_auction,total_ip,most_used_ip,amount_most_used_ip
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,6.0,zm,13.0,et,1.0,1.0,1.166667,20.0,9.194.173.71,4.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,1.0,id,3.0,id,3.0,1.0,1.000000,3.0,45.22.158.84,1.0
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,1.0,id,4.0,id,4.0,1.0,1.000000,4.0,63.67.37.44,1.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,my,1.0,my,1.0,1.0,1.000000,1.0,101.146.132.199,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,2.0,us,154.0,in,1.0,1.0,1.000000,123.0,98.111.158.213,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,5.0,us,645.0,id,3.0,1.0,1.121951,22.0,95.88.86.44,645.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,10.0,us,45.0,ca,1.0,1.0,1.093750,49.0,93.82.160.29,4.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,86.0,zw,809.0,al,1.0,1.0,1.651551,1460.0,98.70.203.119,33.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,1.0,in,5.0,in,5.0,1.0,1.000000,5.0,51.109.135.43,1.0


In [115]:
b = a.groupby(['bidder_id','ip']).size().reset_index()
c = b.groupby(['bidder_id']).min().reset_index()
c = c.rename(columns = {'ip':'rare_ip', 0:'amount_rare_ip'})
c

,bidder_id,rare_ip,amount_rare_ip
0,001068c415025a009fee375a12cff4fcnht8y,139.226.147.115,1
1,002d229ffb247009810828f648afc2ef593rb,37.40.254.131,2
2,0030a2dd87ad2733e0873062e4f83954mkj86,21.67.17.162,1
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,115.47.140.180,1
4,00486a11dff552c4bd7696265724ff81yeo9v,113.39.55.229,1
...,...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,0.0.152.31,1
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,105.144.126.10,1
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,67.61.214.36,1
6612,ffd62646d600b759a985d45918bd6f0431vmz,107.189.39.235,1


In [116]:
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,amount_most_freq_country,rare_country,amount_rare_country,median_country_per_auction,mean_country_per_auction,total_ip,most_used_ip,amount_most_used_ip,rare_ip,amount_rare_ip
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,13.0,et,1.0,1.0,1.166667,20.0,9.194.173.71,4.0,100.75.218.230,1.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,3.0,id,3.0,1.0,1.000000,3.0,45.22.158.84,1.0,19.186.111.152,1.0
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,4.0,id,4.0,1.0,1.000000,4.0,63.67.37.44,1.0,150.248.104.106,1.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,my,1.0,1.0,1.000000,1.0,101.146.132.199,1.0,101.146.132.199,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,154.0,in,1.0,1.0,1.000000,123.0,98.111.158.213,8.0,0.119.34.181,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,645.0,id,3.0,1.0,1.121951,22.0,95.88.86.44,645.0,127.100.35.36,1.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,45.0,ca,1.0,1.0,1.093750,49.0,93.82.160.29,4.0,100.12.82.19,1.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,809.0,al,1.0,1.0,1.651551,1460.0,98.70.203.119,33.0,0.106.114.10,1.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,5.0,in,5.0,1.0,1.000000,5.0,51.109.135.43,1.0,105.125.194.136,1.0


In [117]:
b = a.groupby(['bidder_id','auction','ip']).size().reset_index()
c = b.groupby(['bidder_id','auction']).size().reset_index()
d = c.groupby(['bidder_id']).median().reset_index()
d = d.rename(columns = {0:'median_ip_per_auction'})
d

,bidder_id,median_ip_per_auction
0,001068c415025a009fee375a12cff4fcnht8y,1.0
1,002d229ffb247009810828f648afc2ef593rb,1.0
2,0030a2dd87ad2733e0873062e4f83954mkj86,1.0
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1.0
4,00486a11dff552c4bd7696265724ff81yeo9v,1.0
...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,3.0
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,1.0
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1.0
6612,ffd62646d600b759a985d45918bd6f0431vmz,4.0


In [118]:
train_n_test = pd.merge(train_n_test, d, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,rare_country,amount_rare_country,median_country_per_auction,mean_country_per_auction,total_ip,most_used_ip,amount_most_used_ip,rare_ip,amount_rare_ip,median_ip_per_auction
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,et,1.0,1.0,1.166667,20.0,9.194.173.71,4.0,100.75.218.230,1.0,1.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,id,3.0,1.0,1.000000,3.0,45.22.158.84,1.0,19.186.111.152,1.0,3.0
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,id,4.0,1.0,1.000000,4.0,63.67.37.44,1.0,150.248.104.106,1.0,1.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,my,1.0,1.0,1.000000,1.0,101.146.132.199,1.0,101.146.132.199,1.0,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,in,1.0,1.0,1.000000,123.0,98.111.158.213,8.0,0.119.34.181,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,id,3.0,1.0,1.121951,22.0,95.88.86.44,645.0,127.100.35.36,1.0,1.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,ca,1.0,1.0,1.093750,49.0,93.82.160.29,4.0,100.12.82.19,1.0,1.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,al,1.0,1.0,1.651551,1460.0,98.70.203.119,33.0,0.106.114.10,1.0,2.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,in,5.0,1.0,1.000000,5.0,51.109.135.43,1.0,105.125.194.136,1.0,1.0


In [119]:
b = a.groupby(['bidder_id','auction','ip']).size().reset_index()
c = b.groupby(['bidder_id','auction']).size().reset_index()
d = c.groupby(['bidder_id']).mean().reset_index()
d = d.rename(columns = {0:'mean_ip_per_auction'})
d

,bidder_id,mean_ip_per_auction
0,001068c415025a009fee375a12cff4fcnht8y,1.000000
1,002d229ffb247009810828f648afc2ef593rb,1.000000
2,0030a2dd87ad2733e0873062e4f83954mkj86,1.000000
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1.000000
4,00486a11dff552c4bd7696265724ff81yeo9v,1.538462
...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,33.740973
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,1.333333
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1.000000
6612,ffd62646d600b759a985d45918bd6f0431vmz,7.163636


In [120]:
train_n_test = pd.merge(train_n_test, d, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,amount_rare_country,median_country_per_auction,mean_country_per_auction,total_ip,most_used_ip,amount_most_used_ip,rare_ip,amount_rare_ip,median_ip_per_auction,mean_ip_per_auction
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,1.0,1.0,1.166667,20.0,9.194.173.71,4.0,100.75.218.230,1.0,1.0,1.333333
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,3.0,1.0,1.000000,3.0,45.22.158.84,1.0,19.186.111.152,1.0,3.0,3.000000
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,4.0,1.0,1.000000,4.0,63.67.37.44,1.0,150.248.104.106,1.0,1.0,1.000000
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.0,1.000000,1.0,101.146.132.199,1.0,101.146.132.199,1.0,1.0,1.000000
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,1.0,1.0,1.000000,123.0,98.111.158.213,8.0,0.119.34.181,1.0,2.0,5.608696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,3.0,1.0,1.121951,22.0,95.88.86.44,645.0,127.100.35.36,1.0,1.0,2.170732
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,1.0,1.0,1.093750,49.0,93.82.160.29,4.0,100.12.82.19,1.0,1.0,1.906250
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,1.0,1.0,1.651551,1460.0,98.70.203.119,33.0,0.106.114.10,1.0,2.0,4.754177
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,5.0,1.0,1.000000,5.0,51.109.135.43,1.0,105.125.194.136,1.0,1.0,1.000000


In [121]:
b = a.groupby(['bidder_id','ip']).size().reset_index()
c = b.groupby(['bidder_id']).median().reset_index()
c = c.rename(columns = {0:'median_bid_per_ip'})
c

,bidder_id,median_bid_per_ip
0,001068c415025a009fee375a12cff4fcnht8y,1.0
1,002d229ffb247009810828f648afc2ef593rb,2.0
2,0030a2dd87ad2733e0873062e4f83954mkj86,1.0
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1.0
4,00486a11dff552c4bd7696265724ff81yeo9v,2.0
...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,1.0
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,1.0
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1.0
6612,ffd62646d600b759a985d45918bd6f0431vmz,21.0


In [122]:
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,median_country_per_auction,mean_country_per_auction,total_ip,most_used_ip,amount_most_used_ip,rare_ip,amount_rare_ip,median_ip_per_auction,mean_ip_per_auction,median_bid_per_ip
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,1.0,1.166667,20.0,9.194.173.71,4.0,100.75.218.230,1.0,1.0,1.333333,1.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,1.0,1.000000,3.0,45.22.158.84,1.0,19.186.111.152,1.0,3.0,3.000000,1.0
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,1.0,1.000000,4.0,63.67.37.44,1.0,150.248.104.106,1.0,1.0,1.000000,1.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.000000,1.0,101.146.132.199,1.0,101.146.132.199,1.0,1.0,1.000000,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,1.0,1.000000,123.0,98.111.158.213,8.0,0.119.34.181,1.0,2.0,5.608696,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,1.0,1.121951,22.0,95.88.86.44,645.0,127.100.35.36,1.0,1.0,2.170732,5.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,1.0,1.093750,49.0,93.82.160.29,4.0,100.12.82.19,1.0,1.0,1.906250,1.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,1.0,1.651551,1460.0,98.70.203.119,33.0,0.106.114.10,1.0,2.0,4.754177,1.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,1.0,1.000000,5.0,51.109.135.43,1.0,105.125.194.136,1.0,1.0,1.000000,1.0


In [123]:
b = a.groupby(['bidder_id','ip']).size().reset_index()
c = b.groupby(['bidder_id']).mean().reset_index()
c = c.rename(columns = {0:'mean_bid_per_ip'})
c

,bidder_id,mean_bid_per_ip
0,001068c415025a009fee375a12cff4fcnht8y,1.000000
1,002d229ffb247009810828f648afc2ef593rb,2.000000
2,0030a2dd87ad2733e0873062e4f83954mkj86,1.000000
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1.000000
4,00486a11dff552c4bd7696265724ff81yeo9v,2.000000
...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,1.339047
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,1.222222
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1.000000
6612,ffd62646d600b759a985d45918bd6f0431vmz,17.945946


In [124]:
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,mean_country_per_auction,total_ip,most_used_ip,amount_most_used_ip,rare_ip,amount_rare_ip,median_ip_per_auction,mean_ip_per_auction,median_bid_per_ip,mean_bid_per_ip
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,1.166667,20.0,9.194.173.71,4.0,100.75.218.230,1.0,1.0,1.333333,1.0,1.200000
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,1.000000,3.0,45.22.158.84,1.0,19.186.111.152,1.0,3.0,3.000000,1.0,1.000000
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,1.000000,4.0,63.67.37.44,1.0,150.248.104.106,1.0,1.0,1.000000,1.0,1.000000
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,1.000000,1.0,101.146.132.199,1.0,101.146.132.199,1.0,1.0,1.000000,1.0,1.000000
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,1.000000,123.0,98.111.158.213,8.0,0.119.34.181,1.0,2.0,5.608696,1.0,1.260163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,1.121951,22.0,95.88.86.44,645.0,127.100.35.36,1.0,1.0,2.170732,5.0,44.681818
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,1.093750,49.0,93.82.160.29,4.0,100.12.82.19,1.0,1.0,1.906250,1.0,1.346939
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,1.651551,1460.0,98.70.203.119,33.0,0.106.114.10,1.0,2.0,4.754177,1.0,1.480822
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,1.000000,5.0,51.109.135.43,1.0,105.125.194.136,1.0,1.0,1.000000,1.0,1.000000


### Ratio of sharing ip / unique ip
- calculate ratio the bidder used shared ips / unique ips

In [ ]:
# Ratio of sharing ip / unique ip
b = a.groupby(['ip']).bidder_id.nunique().reset_index()
b = b.rename(columns = {'bidder_id':'ip_shared_by_many'})
b

In [126]:
b['ip_shared_by_many'][b['ip_shared_by_many'] > 1] = 'many'

D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [127]:
b['ip_shared_by_many'][b['ip_shared_by_many'] == 1] = 'only_one'

In [128]:
b

,ip,ip_shared_by_many
0,0.0.101.184,many
1,0.0.101.48,only_one
2,0.0.113.26,only_one
3,0.0.129.216,only_one
4,0.0.137.132,only_one
...,...,...
2303986,99.99.70.251,only_one
2303987,99.99.72.22,only_one
2303988,99.99.79.36,only_one
2303989,99.99.91.35,only_one


In [129]:
bids_with_ip = pd.merge(bids, b, on='ip', how='left')
bids_with_ip

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url,ip_shared_by_many
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,1970-04-23 22:54:03.157894736,us,69.166.231.58,vasstdc27m7nks3,many
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,1970-04-23 22:54:03.157894736,in,50.201.125.84,jmqlhflrzwuay9c,many
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,1970-04-23 22:54:03.157894736,py,112.54.208.157,vasstdc27m7nks3,only_one
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,1970-04-23 22:54:03.157894736,in,18.99.175.133,vasstdc27m7nks3,many
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,1970-04-23 22:54:03.157894736,in,145.138.5.37,vasstdc27m7nks3,only_one
...,...,...,...,...,...,...,...,...,...,...
7656329,7656329,626159dd6f2228ede002d9f9340f75b7puk8d,3e64w,jewelry,phone91,1970-04-23 09:00:22.052631578,ru,140.204.227.63,cghhmomsaxi6pug,many
7656330,7656330,a318ea333ceee1ba39a494476386136a826dv,xn0y0,mobile,phone236,1970-04-23 09:00:22.052631578,pl,24.232.159.118,wgggpdg2gx5pesn,many
7656331,7656331,f5b2bbad20d1d7ded3ed960393bec0f40u6hn,gja6c,sporting goods,phone80,1970-04-23 09:00:22.052631578,za,80.237.28.246,5xgysg14grlersa,only_one
7656332,7656332,d4bd412590f5106b9d887a43c51b254eldo4f,hmwk8,jewelry,phone349,1970-04-23 09:00:22.052631578,my,91.162.27.152,bhtrek44bzi2wfl,many


In [130]:
c = bids_with_ip.groupby(['bidder_id','ip_shared_by_many']).size().reset_index()
c = c.rename(columns = {0: 'counts'})
c

,bidder_id,ip_shared_by_many,counts
0,001068c415025a009fee375a12cff4fcnht8y,many,1
1,002d229ffb247009810828f648afc2ef593rb,only_one,2
2,0030a2dd87ad2733e0873062e4f83954mkj86,many,1
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,many,3
4,00486a11dff552c4bd7696265724ff81yeo9v,many,20
...,...,...,...
10912,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,many,20
10913,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,only_one,2
10914,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,only_one,1
10915,ffd62646d600b759a985d45918bd6f0431vmz,many,664


In [131]:
d = c.pivot('bidder_id', 'ip_shared_by_many', 'counts').reset_index().fillna(0)
d

ip_shared_by_many,bidder_id,many,only_one
0,001068c415025a009fee375a12cff4fcnht8y,1.0,0.0
1,002d229ffb247009810828f648afc2ef593rb,0.0,2.0
2,0030a2dd87ad2733e0873062e4f83954mkj86,1.0,0.0
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,3.0,0.0
4,00486a11dff552c4bd7696265724ff81yeo9v,20.0,0.0
...,...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,12344.0,12731.0
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,20.0,2.0
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,0.0,1.0
6612,ffd62646d600b759a985d45918bd6f0431vmz,664.0,0.0


In [132]:
d.iloc[:,1:] = d.iloc[:,1:].div(d.iloc[:,1:].sum(axis=1), axis=0)

In [133]:
d = d.rename(columns ={'many':'ratio_used_shared_ip', 'only_one':'ratio_used_unique_ip'})
d

ip_shared_by_many,bidder_id,ratio_used_shared_ip,ratio_used_unique_ip
0,001068c415025a009fee375a12cff4fcnht8y,1.000000,0.000000
1,002d229ffb247009810828f648afc2ef593rb,0.000000,1.000000
2,0030a2dd87ad2733e0873062e4f83954mkj86,1.000000,0.000000
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1.000000,0.000000
4,00486a11dff552c4bd7696265724ff81yeo9v,1.000000,0.000000
...,...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,0.492283,0.507717
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,0.909091,0.090909
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,0.000000,1.000000
6612,ffd62646d600b759a985d45918bd6f0431vmz,1.000000,0.000000


In [134]:
d = d.drop(['ratio_used_shared_ip'], axis=1)
train_n_test = pd.merge(train_n_test, d, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,total_ip,most_used_ip,amount_most_used_ip,rare_ip,amount_rare_ip,median_ip_per_auction,mean_ip_per_auction,median_bid_per_ip,mean_bid_per_ip,ratio_used_unique_ip
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,20.0,9.194.173.71,4.0,100.75.218.230,1.0,1.0,1.333333,1.0,1.200000,0.250000
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,3.0,45.22.158.84,1.0,19.186.111.152,1.0,3.0,3.000000,1.0,1.000000,1.000000
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,4.0,63.67.37.44,1.0,150.248.104.106,1.0,1.0,1.000000,1.0,1.000000,0.000000
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,101.146.132.199,1.0,101.146.132.199,1.0,1.0,1.000000,1.0,1.000000,0.000000
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,123.0,98.111.158.213,8.0,0.119.34.181,1.0,2.0,5.608696,1.0,1.260163,0.477419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,22.0,95.88.86.44,645.0,127.100.35.36,1.0,1.0,2.170732,5.0,44.681818,0.003052
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,49.0,93.82.160.29,4.0,100.12.82.19,1.0,1.0,1.906250,1.0,1.346939,0.166667
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,1460.0,98.70.203.119,33.0,0.106.114.10,1.0,2.0,4.754177,1.0,1.480822,0.208603
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,5.0,51.109.135.43,1.0,105.125.194.136,1.0,1.0,1.000000,1.0,1.000000,0.200000


### Delta time 5
- calculate time difference between two bids of bidder placed by using same ip
- calculate median and min time difference between two bids on same ip

In [135]:
# Difference time between two bids on same ip
a = bids[['bidder_id', 'time', 'ip']]
a = a.sort_values(['bidder_id', 'time']).reset_index()
a

,index,bidder_id,time,ip
0,7179832,001068c415025a009fee375a12cff4fcnht8y,1970-04-23 08:12:25.052631578,139.226.147.115
1,1281292,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.105263157,37.40.254.131
2,1281311,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.210526315,37.40.254.131
3,6805028,0030a2dd87ad2733e0873062e4f83954mkj86,1970-04-23 07:42:33.947368421,21.67.17.162
4,3967330,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1970-04-22 13:46:58.631578947,44.241.8.179
...,...,...,...,...
7656329,2336021,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:02.947368421,214.13.99.0
7656330,2337626,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:11.421052631,144.253.142.62
7656331,2344645,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:50.052631578,166.67.25.123
7656332,2350507,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:41:21.631578947,214.13.99.0


In [136]:
b = a.drop(['index'], axis=1)
b['bidder_id_prev'] = pd.Series(np.append([np.nan], b.bidder_id.values[0:-1]))
b['time_prev'] = pd.Series(np.append([pd.NaT], b.time.values[0:-1]))
b['time_prev'] = pd.to_datetime(b['time_prev'])
b['ip_prev'] = pd.Series(np.append([np.nan], b.ip.values[0:-1]))
b

,bidder_id,time,ip,bidder_id_prev,time_prev,ip_prev
0,001068c415025a009fee375a12cff4fcnht8y,1970-04-23 08:12:25.052631578,139.226.147.115,NaN,NaT,NaN
1,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.105263157,37.40.254.131,001068c415025a009fee375a12cff4fcnht8y,1970-04-23 08:12:25.052631578,139.226.147.115
2,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.210526315,37.40.254.131,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.105263157,37.40.254.131
3,0030a2dd87ad2733e0873062e4f83954mkj86,1970-04-23 07:42:33.947368421,21.67.17.162,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.210526315,37.40.254.131
4,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1970-04-22 13:46:58.631578947,44.241.8.179,0030a2dd87ad2733e0873062e4f83954mkj86,1970-04-23 07:42:33.947368421,21.67.17.162
...,...,...,...,...,...,...
7656329,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:02.947368421,214.13.99.0,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:39:57.315789473,214.13.99.0
7656330,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:11.421052631,144.253.142.62,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:02.947368421,214.13.99.0
7656331,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:50.052631578,166.67.25.123,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:11.421052631,144.253.142.62
7656332,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:41:21.631578947,214.13.99.0,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:50.052631578,166.67.25.123


In [137]:
b['dt_time'] = b['time'] - b['time_prev']
b['dt_time'] = b['dt_time'].dt.total_seconds()
b

,bidder_id,time,ip,bidder_id_prev,time_prev,ip_prev,dt_time
0,001068c415025a009fee375a12cff4fcnht8y,1970-04-23 08:12:25.052631578,139.226.147.115,NaN,NaT,NaN,NaN
1,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.105263157,37.40.254.131,001068c415025a009fee375a12cff4fcnht8y,1970-04-23 08:12:25.052631578,139.226.147.115,60399.052632
2,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.210526315,37.40.254.131,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.105263157,37.40.254.131,0.105263
3,0030a2dd87ad2733e0873062e4f83954mkj86,1970-04-23 07:42:33.947368421,21.67.17.162,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.210526315,37.40.254.131,-62190.263158
4,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1970-04-22 13:46:58.631578947,44.241.8.179,0030a2dd87ad2733e0873062e4f83954mkj86,1970-04-23 07:42:33.947368421,21.67.17.162,-64535.315789
...,...,...,...,...,...,...,...
7656329,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:02.947368421,214.13.99.0,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:39:57.315789473,214.13.99.0,5.631579
7656330,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:11.421052631,144.253.142.62,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:02.947368421,214.13.99.0,8.473684
7656331,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:50.052631578,166.67.25.123,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:11.421052631,144.253.142.62,38.631579
7656332,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:41:21.631578947,214.13.99.0,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:50.052631578,166.67.25.123,31.578947


In [138]:
c = b[b.bidder_id == b.bidder_id_prev][b.ip == b.ip_prev][b.dt_time > 0.0]
c = c.reset_index()
c

D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,index,bidder_id,time,ip,bidder_id_prev,time_prev,ip_prev,dt_time
0,2,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.210526315,37.40.254.131,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.105263157,37.40.254.131,0.105263
1,8,00486a11dff552c4bd7696265724ff81yeo9v,1970-04-22 11:44:01.157894736,143.118.40.162,00486a11dff552c4bd7696265724ff81yeo9v,1970-04-22 11:43:56.526315789,143.118.40.162,4.631579
2,13,00486a11dff552c4bd7696265724ff81yeo9v,1970-04-22 13:13:13.684210526,236.63.15.129,00486a11dff552c4bd7696265724ff81yeo9v,1970-04-22 13:13:13.315789473,236.63.15.129,0.368421
3,15,00486a11dff552c4bd7696265724ff81yeo9v,1970-04-22 13:28:35.315789473,220.193.41.160,00486a11dff552c4bd7696265724ff81yeo9v,1970-04-22 13:28:31.421052631,220.193.41.160,3.894737
4,18,00486a11dff552c4bd7696265724ff81yeo9v,1970-04-22 14:38:22.000000000,224.192.40.189,00486a11dff552c4bd7696265724ff81yeo9v,1970-04-22 14:38:15.000000000,224.192.40.189,7.000000
...,...,...,...,...,...,...,...,...
311194,7656282,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:25:48.894736842,47.26.163.208,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:25:26.684210526,47.26.163.208,22.210526
311195,7656305,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:33:24.052631578,177.47.141.241,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:33:09.157894736,177.47.141.241,14.894737
311196,7656317,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:36:21.210526315,150.248.104.106,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:36:08.894736842,150.248.104.106,12.315789
311197,7656324,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:38:52.000000000,144.253.142.62,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:38:18.842105263,144.253.142.62,33.157895


In [139]:
d = c.groupby('bidder_id').dt_time.median().reset_index()
d = d.rename(columns = {'dt_time': 'median_dt_time_same_ip'})
d

,bidder_id,median_dt_time_same_ip
0,002d229ffb247009810828f648afc2ef593rb,0.105263
1,00486a11dff552c4bd7696265724ff81yeo9v,4.631579
2,0051aef3fdeacdadba664b9b3b07e04e4coc6,1.894737
3,0053b78cde37c4384a20d2da9aa4272aym4pb,0.421053
4,0061edfc5b07ff3d70d693883a38d370oy4fs,2.052632
...,...,...
3986,ffa7b0b0f144b1594131d99e50c17a0bwbym3,1.131579
3987,ffacbed056cbfaa60c1fcf51f0d381bddr3ly,2.736842
3988,ffbc0fdfbf19a8a9116b68714138f2902cc13,0.578947
3989,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,3.842105


In [140]:
train_n_test = pd.merge(train_n_test, d, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,most_used_ip,amount_most_used_ip,rare_ip,amount_rare_ip,median_ip_per_auction,mean_ip_per_auction,median_bid_per_ip,mean_bid_per_ip,ratio_used_unique_ip,median_dt_time_same_ip
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,9.194.173.71,4.0,100.75.218.230,1.0,1.0,1.333333,1.0,1.200000,0.250000,NaN
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,45.22.158.84,1.0,19.186.111.152,1.0,3.0,3.000000,1.0,1.000000,1.000000,NaN
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,63.67.37.44,1.0,150.248.104.106,1.0,1.0,1.000000,1.0,1.000000,0.000000,NaN
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,101.146.132.199,1.0,101.146.132.199,1.0,1.0,1.000000,1.0,1.000000,0.000000,NaN
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,98.111.158.213,8.0,0.119.34.181,1.0,2.0,5.608696,1.0,1.260163,0.477419,2.157895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,95.88.86.44,645.0,127.100.35.36,1.0,1.0,2.170732,5.0,44.681818,0.003052,1.421053
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,93.82.160.29,4.0,100.12.82.19,1.0,1.0,1.906250,1.0,1.346939,0.166667,5.973684
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,98.70.203.119,33.0,0.106.114.10,1.0,2.0,4.754177,1.0,1.480822,0.208603,3.184211
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,51.109.135.43,1.0,105.125.194.136,1.0,1.0,1.000000,1.0,1.000000,0.200000,NaN


In [141]:
d = c.groupby('bidder_id').dt_time.min().reset_index()
d = d.rename(columns = {'dt_time': 'min_dt_time_same_ip'})
d

,bidder_id,min_dt_time_same_ip
0,002d229ffb247009810828f648afc2ef593rb,0.105263
1,00486a11dff552c4bd7696265724ff81yeo9v,0.368421
2,0051aef3fdeacdadba664b9b3b07e04e4coc6,0.052632
3,0053b78cde37c4384a20d2da9aa4272aym4pb,0.052632
4,0061edfc5b07ff3d70d693883a38d370oy4fs,0.157895
...,...,...
3986,ffa7b0b0f144b1594131d99e50c17a0bwbym3,0.368421
3987,ffacbed056cbfaa60c1fcf51f0d381bddr3ly,2.736842
3988,ffbc0fdfbf19a8a9116b68714138f2902cc13,0.052632
3989,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,1.368421


In [142]:
train_n_test = pd.merge(train_n_test, d, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,amount_most_used_ip,rare_ip,amount_rare_ip,median_ip_per_auction,mean_ip_per_auction,median_bid_per_ip,mean_bid_per_ip,ratio_used_unique_ip,median_dt_time_same_ip,min_dt_time_same_ip
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,4.0,100.75.218.230,1.0,1.0,1.333333,1.0,1.200000,0.250000,NaN,NaN
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,1.0,19.186.111.152,1.0,3.0,3.000000,1.0,1.000000,1.000000,NaN,NaN
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,1.0,150.248.104.106,1.0,1.0,1.000000,1.0,1.000000,0.000000,NaN,NaN
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,101.146.132.199,1.0,1.0,1.000000,1.0,1.000000,0.000000,NaN,NaN
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,8.0,0.119.34.181,1.0,2.0,5.608696,1.0,1.260163,0.477419,2.157895,0.789474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,645.0,127.100.35.36,1.0,1.0,2.170732,5.0,44.681818,0.003052,1.421053,0.052632
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,4.0,100.12.82.19,1.0,1.0,1.906250,1.0,1.346939,0.166667,5.973684,0.526316
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,33.0,0.106.114.10,1.0,2.0,4.754177,1.0,1.480822,0.208603,3.184211,0.157895
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,1.0,105.125.194.136,1.0,1.0,1.000000,1.0,1.000000,0.200000,NaN,NaN


### Delta time 6
- calculate time difference between two bids of bidder placed by using different ips
- calculate median and min time difference between two bids on different ips

In [143]:
# Difference time between two bids on different ip
a = bids[['bidder_id', 'time', 'ip']]
a = a.sort_values(['bidder_id', 'time']).reset_index()
b = a.drop(['index'], axis=1)
b['bidder_id_prev'] = pd.Series(np.append([np.nan], b.bidder_id.values[0:-1]))
b['time_prev'] = pd.Series(np.append([pd.NaT], b.time.values[0:-1]))
b['time_prev'] = pd.to_datetime(b['time_prev'])
b['ip_prev'] = pd.Series(np.append([np.nan], b.ip.values[0:-1]))
b

,bidder_id,time,ip,bidder_id_prev,time_prev,ip_prev
0,001068c415025a009fee375a12cff4fcnht8y,1970-04-23 08:12:25.052631578,139.226.147.115,NaN,NaT,NaN
1,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.105263157,37.40.254.131,001068c415025a009fee375a12cff4fcnht8y,1970-04-23 08:12:25.052631578,139.226.147.115
2,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.210526315,37.40.254.131,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.105263157,37.40.254.131
3,0030a2dd87ad2733e0873062e4f83954mkj86,1970-04-23 07:42:33.947368421,21.67.17.162,002d229ffb247009810828f648afc2ef593rb,1970-04-24 00:59:04.210526315,37.40.254.131
4,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1970-04-22 13:46:58.631578947,44.241.8.179,0030a2dd87ad2733e0873062e4f83954mkj86,1970-04-23 07:42:33.947368421,21.67.17.162
...,...,...,...,...,...,...
7656329,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:02.947368421,214.13.99.0,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:39:57.315789473,214.13.99.0
7656330,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:11.421052631,144.253.142.62,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:02.947368421,214.13.99.0
7656331,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:50.052631578,166.67.25.123,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:11.421052631,144.253.142.62
7656332,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:41:21.631578947,214.13.99.0,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:50.052631578,166.67.25.123


In [144]:
b['dt_time'] = b['time'] - b['time_prev']
b['dt_time'] = b['dt_time'].dt.total_seconds()
c = b[b.bidder_id == b.bidder_id_prev][b.ip != b.ip_prev][b.dt_time > 0.0]
c = c.reset_index()
c

D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,index,bidder_id,time,ip,bidder_id_prev,time_prev,ip_prev,dt_time
0,5,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1970-04-23 06:36:45.052631578,190.88.89.83,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1970-04-22 13:46:58.631578947,44.241.8.179,60586.421053
1,6,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1970-04-23 08:06:14.315789473,115.47.140.180,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1970-04-23 06:36:45.052631578,190.88.89.83,5369.263158
2,9,00486a11dff552c4bd7696265724ff81yeo9v,1970-04-22 11:55:39.684210526,54.212.177.220,00486a11dff552c4bd7696265724ff81yeo9v,1970-04-22 11:44:01.157894736,143.118.40.162,698.526316
3,10,00486a11dff552c4bd7696265724ff81yeo9v,1970-04-22 12:30:39.947368421,236.63.15.129,00486a11dff552c4bd7696265724ff81yeo9v,1970-04-22 11:55:39.684210526,54.212.177.220,2100.263158
4,11,00486a11dff552c4bd7696265724ff81yeo9v,1970-04-22 12:48:21.894736842,127.247.172.237,00486a11dff552c4bd7696265724ff81yeo9v,1970-04-22 12:30:39.947368421,236.63.15.129,1061.947368
...,...,...,...,...,...,...,...,...
6525700,7656327,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:39:57.105263157,46.186.121.224,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:39:55.842105263,214.13.99.0,1.263158
6525701,7656328,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:39:57.315789473,214.13.99.0,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:39:57.105263157,46.186.121.224,0.210526
6525702,7656330,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:11.421052631,144.253.142.62,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:02.947368421,214.13.99.0,8.473684
6525703,7656331,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:50.052631578,166.67.25.123,ffd62646d600b759a985d45918bd6f0431vmz,1970-04-24 02:40:11.421052631,144.253.142.62,38.631579


In [145]:
c = c.drop(['index'], axis=1)
d = c.groupby('bidder_id').dt_time.median().reset_index()
d = d.rename(columns = {'dt_time': 'median_dt_time_other_ip'})
d

,bidder_id,median_dt_time_other_ip
0,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,32977.842105
1,00486a11dff552c4bd7696265724ff81yeo9v,1427.947368
2,0051aef3fdeacdadba664b9b3b07e04e4coc6,144.552632
3,0053b78cde37c4384a20d2da9aa4272aym4pb,0.421053
4,0061edfc5b07ff3d70d693883a38d370oy4fs,59.368421
...,...,...
5371,ffacbed056cbfaa60c1fcf51f0d381bddr3ly,2142.684211
5372,ffaf0a972a6dcb3910fd6b16045781e2ava5y,4388.157895
5373,ffbc0fdfbf19a8a9116b68714138f2902cc13,0.368421
5374,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,1387.631579


In [146]:
train_n_test = pd.merge(train_n_test, d, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,rare_ip,amount_rare_ip,median_ip_per_auction,mean_ip_per_auction,median_bid_per_ip,mean_bid_per_ip,ratio_used_unique_ip,median_dt_time_same_ip,min_dt_time_same_ip,median_dt_time_other_ip
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,100.75.218.230,1.0,1.0,1.333333,1.0,1.200000,0.250000,NaN,NaN,345.842105
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,19.186.111.152,1.0,3.0,3.000000,1.0,1.000000,1.000000,NaN,NaN,3233.578947
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,150.248.104.106,1.0,1.0,1.000000,1.0,1.000000,0.000000,NaN,NaN,2532.052632
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,101.146.132.199,1.0,1.0,1.000000,1.0,1.000000,0.000000,NaN,NaN,NaN
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,0.119.34.181,1.0,2.0,5.608696,1.0,1.260163,0.477419,2.157895,0.789474,15.473684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,127.100.35.36,1.0,1.0,2.170732,5.0,44.681818,0.003052,1.421053,0.052632,1.526316
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,100.12.82.19,1.0,1.0,1.906250,1.0,1.346939,0.166667,5.973684,0.526316,152.000000
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,0.106.114.10,1.0,2.0,4.754177,1.0,1.480822,0.208603,3.184211,0.157895,6.894737
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,105.125.194.136,1.0,1.0,1.000000,1.0,1.000000,0.200000,NaN,NaN,1519.736842


In [147]:
d = c.groupby('bidder_id').dt_time.min().reset_index()
d = d.rename(columns = {'dt_time': 'min_dt_time_other_ip'})
d

,bidder_id,min_dt_time_other_ip
0,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,5369.263158
1,00486a11dff552c4bd7696265724ff81yeo9v,40.421053
2,0051aef3fdeacdadba664b9b3b07e04e4coc6,3.263158
3,0053b78cde37c4384a20d2da9aa4272aym4pb,0.052632
4,0061edfc5b07ff3d70d693883a38d370oy4fs,0.315789
...,...,...
5371,ffacbed056cbfaa60c1fcf51f0d381bddr3ly,18.578947
5372,ffaf0a972a6dcb3910fd6b16045781e2ava5y,3448.473684
5373,ffbc0fdfbf19a8a9116b68714138f2902cc13,0.052632
5374,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,7.052632


In [148]:
train_n_test = pd.merge(train_n_test, d, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,amount_rare_ip,median_ip_per_auction,mean_ip_per_auction,median_bid_per_ip,mean_bid_per_ip,ratio_used_unique_ip,median_dt_time_same_ip,min_dt_time_same_ip,median_dt_time_other_ip,min_dt_time_other_ip
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,1.0,1.0,1.333333,1.0,1.200000,0.250000,NaN,NaN,345.842105,5.210526
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,1.0,3.0,3.000000,1.0,1.000000,1.000000,NaN,NaN,3233.578947,1989.315789
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,1.0,1.0,1.000000,1.0,1.000000,0.000000,NaN,NaN,2532.052632,1450.842105
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.0,1.000000,1.0,1.000000,0.000000,NaN,NaN,NaN,NaN
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,1.0,2.0,5.608696,1.0,1.260163,0.477419,2.157895,0.789474,15.473684,0.052632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,1.0,1.0,2.170732,5.0,44.681818,0.003052,1.421053,0.052632,1.526316,0.052632
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,1.0,1.0,1.906250,1.0,1.346939,0.166667,5.973684,0.526316,152.000000,0.473684
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,1.0,2.0,4.754177,1.0,1.480822,0.208603,3.184211,0.157895,6.894737,0.052632
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,1.0,1.0,1.000000,1.0,1.000000,0.200000,NaN,NaN,1519.736842,459.684211


### URL Features
- calculate total urls, most used url, rare url used by bidder
- calculate median and mean number of bids per url
- calculate median and mean number of urls per auction

In [149]:
# Total url, most used url, rare url, median and mean bid per url
a = bids[['bidder_id','auction','url']]
b = a.groupby(['bidder_id','url']).size().reset_index()
c = b.groupby(['bidder_id']).size().reset_index()
c = c.rename(columns = {0:'total_url'})
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,median_ip_per_auction,mean_ip_per_auction,median_bid_per_ip,mean_bid_per_ip,ratio_used_unique_ip,median_dt_time_same_ip,min_dt_time_same_ip,median_dt_time_other_ip,min_dt_time_other_ip,total_url
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,1.0,1.333333,1.0,1.200000,0.250000,NaN,NaN,345.842105,5.210526,1.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,3.0,3.000000,1.0,1.000000,1.000000,NaN,NaN,3233.578947,1989.315789,2.0
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,1.0,1.000000,1.0,1.000000,0.000000,NaN,NaN,2532.052632,1450.842105,2.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.000000,1.0,1.000000,0.000000,NaN,NaN,NaN,NaN,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,2.0,5.608696,1.0,1.260163,0.477419,2.157895,0.789474,15.473684,0.052632,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,1.0,2.170732,5.0,44.681818,0.003052,1.421053,0.052632,1.526316,0.052632,4.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,1.0,1.906250,1.0,1.346939,0.166667,5.973684,0.526316,152.000000,0.473684,18.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,2.0,4.754177,1.0,1.480822,0.208603,3.184211,0.157895,6.894737,0.052632,1049.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,1.0,1.000000,1.0,1.000000,0.200000,NaN,NaN,1519.736842,459.684211,2.0


In [150]:
b = a.groupby(['bidder_id','url']).size().reset_index()
c = b.groupby(['bidder_id']).max().reset_index()
c = c.rename(columns = {'url':'most_used_url', 0:'amount_most_used_url'})
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,median_bid_per_ip,mean_bid_per_ip,ratio_used_unique_ip,median_dt_time_same_ip,min_dt_time_same_ip,median_dt_time_other_ip,min_dt_time_other_ip,total_url,most_used_url,amount_most_used_url
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,1.0,1.200000,0.250000,NaN,NaN,345.842105,5.210526,1.0,vasstdc27m7nks3,24.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,1.0,1.000000,1.000000,NaN,NaN,3233.578947,1989.315789,2.0,vgbf6zjir47a9y8,2.0
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,1.0,1.000000,0.000000,NaN,NaN,2532.052632,1450.842105,2.0,vasstdc27m7nks3,3.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.000000,0.000000,NaN,NaN,NaN,NaN,1.0,vasstdc27m7nks3,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,1.0,1.260163,0.477419,2.157895,0.789474,15.473684,0.052632,91.0,zui5a6sfx0y76bi,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,5.0,44.681818,0.003052,1.421053,0.052632,1.526316,0.052632,4.0,zo1ifv9d3ocwksy,734.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,1.0,1.346939,0.166667,5.973684,0.526316,152.000000,0.473684,18.0,yv41qqhhwlf2nfg,32.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,1.0,1.480822,0.208603,3.184211,0.157895,6.894737,0.052632,1049.0,zz3d6spn0i68nhz,725.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,1.0,1.000000,0.200000,NaN,NaN,1519.736842,459.684211,2.0,vasstdc27m7nks3,3.0


In [151]:
b = a.groupby(['bidder_id','url']).size().reset_index()
c = b.groupby(['bidder_id']).min().reset_index()
c = c.rename(columns = {'url':'rare_url', 0:'amount_rare_url'})
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,ratio_used_unique_ip,median_dt_time_same_ip,min_dt_time_same_ip,median_dt_time_other_ip,min_dt_time_other_ip,total_url,most_used_url,amount_most_used_url,rare_url,amount_rare_url
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,0.250000,NaN,NaN,345.842105,5.210526,1.0,vasstdc27m7nks3,24.0,vasstdc27m7nks3,24.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,1.000000,NaN,NaN,3233.578947,1989.315789,2.0,vgbf6zjir47a9y8,2.0,vasstdc27m7nks3,1.0
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,0.000000,NaN,NaN,2532.052632,1450.842105,2.0,vasstdc27m7nks3,3.0,i7m2pm7a1w52dqd,1.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,0.000000,NaN,NaN,NaN,NaN,1.0,vasstdc27m7nks3,1.0,vasstdc27m7nks3,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,0.477419,2.157895,0.789474,15.473684,0.052632,91.0,zui5a6sfx0y76bi,54.0,0dja032dfcct6e8,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,0.003052,1.421053,0.052632,1.526316,0.052632,4.0,zo1ifv9d3ocwksy,734.0,g8e8izd33vphs6r,1.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,0.166667,5.973684,0.526316,152.000000,0.473684,18.0,yv41qqhhwlf2nfg,32.0,16cy2e1d5jl7igv,1.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,0.208603,3.184211,0.157895,6.894737,0.052632,1049.0,zz3d6spn0i68nhz,725.0,04vxqqwddrvyw8j,1.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,0.200000,NaN,NaN,1519.736842,459.684211,2.0,vasstdc27m7nks3,3.0,umq82yct2iei4f4,2.0


In [152]:
b = a.groupby(['bidder_id','url']).size().reset_index()
c = b.groupby(['bidder_id']).median().reset_index()
c = c.rename(columns = {0:'median_bid_per_url'})
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,median_dt_time_same_ip,min_dt_time_same_ip,median_dt_time_other_ip,min_dt_time_other_ip,total_url,most_used_url,amount_most_used_url,rare_url,amount_rare_url,median_bid_per_url
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,NaN,NaN,345.842105,5.210526,1.0,vasstdc27m7nks3,24.0,vasstdc27m7nks3,24.0,24.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,NaN,NaN,3233.578947,1989.315789,2.0,vgbf6zjir47a9y8,2.0,vasstdc27m7nks3,1.0,1.5
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,NaN,NaN,2532.052632,1450.842105,2.0,vasstdc27m7nks3,3.0,i7m2pm7a1w52dqd,1.0,2.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,NaN,NaN,NaN,NaN,1.0,vasstdc27m7nks3,1.0,vasstdc27m7nks3,1.0,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,2.157895,0.789474,15.473684,0.052632,91.0,zui5a6sfx0y76bi,54.0,0dja032dfcct6e8,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,1.421053,0.052632,1.526316,0.052632,4.0,zo1ifv9d3ocwksy,734.0,g8e8izd33vphs6r,1.0,124.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,5.973684,0.526316,152.000000,0.473684,18.0,yv41qqhhwlf2nfg,32.0,16cy2e1d5jl7igv,1.0,1.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,3.184211,0.157895,6.894737,0.052632,1049.0,zz3d6spn0i68nhz,725.0,04vxqqwddrvyw8j,1.0,1.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,NaN,NaN,1519.736842,459.684211,2.0,vasstdc27m7nks3,3.0,umq82yct2iei4f4,2.0,2.5


In [153]:
b = a.groupby(['bidder_id','url']).size().reset_index()
c = b.groupby(['bidder_id']).mean().reset_index()
c = c.rename(columns = {0:'mean_bid_per_url'})
train_n_test = pd.merge(train_n_test, c, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,min_dt_time_same_ip,median_dt_time_other_ip,min_dt_time_other_ip,total_url,most_used_url,amount_most_used_url,rare_url,amount_rare_url,median_bid_per_url,mean_bid_per_url
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,NaN,345.842105,5.210526,1.0,vasstdc27m7nks3,24.0,vasstdc27m7nks3,24.0,24.0,24.000000
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,NaN,3233.578947,1989.315789,2.0,vgbf6zjir47a9y8,2.0,vasstdc27m7nks3,1.0,1.5,1.500000
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,NaN,2532.052632,1450.842105,2.0,vasstdc27m7nks3,3.0,i7m2pm7a1w52dqd,1.0,2.0,2.000000
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,NaN,NaN,NaN,1.0,vasstdc27m7nks3,1.0,vasstdc27m7nks3,1.0,1.0,1.000000
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,0.789474,15.473684,0.052632,91.0,zui5a6sfx0y76bi,54.0,0dja032dfcct6e8,1.0,1.0,1.703297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,0.052632,1.526316,0.052632,4.0,zo1ifv9d3ocwksy,734.0,g8e8izd33vphs6r,1.0,124.0,245.750000
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,0.526316,152.000000,0.473684,18.0,yv41qqhhwlf2nfg,32.0,16cy2e1d5jl7igv,1.0,1.0,3.666667
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,0.157895,6.894737,0.052632,1049.0,zz3d6spn0i68nhz,725.0,04vxqqwddrvyw8j,1.0,1.0,2.061010
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,NaN,1519.736842,459.684211,2.0,vasstdc27m7nks3,3.0,umq82yct2iei4f4,2.0,2.5,2.500000


In [154]:
b = a.groupby(['bidder_id','auction','url']).size().reset_index()
c = b.groupby(['bidder_id','auction']).size().reset_index()
d = c.groupby(['bidder_id']).median().reset_index()
d = d.rename(columns = {0:'median_url_per_auction'})
train_n_test = pd.merge(train_n_test, d, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,median_dt_time_other_ip,min_dt_time_other_ip,total_url,most_used_url,amount_most_used_url,rare_url,amount_rare_url,median_bid_per_url,mean_bid_per_url,median_url_per_auction
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,345.842105,5.210526,1.0,vasstdc27m7nks3,24.0,vasstdc27m7nks3,24.0,24.0,24.000000,1.0
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,3233.578947,1989.315789,2.0,vgbf6zjir47a9y8,2.0,vasstdc27m7nks3,1.0,1.5,1.500000,2.0
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,2532.052632,1450.842105,2.0,vasstdc27m7nks3,3.0,i7m2pm7a1w52dqd,1.0,2.0,2.000000,1.0
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,NaN,NaN,1.0,vasstdc27m7nks3,1.0,vasstdc27m7nks3,1.0,1.0,1.000000,1.0
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,15.473684,0.052632,91.0,zui5a6sfx0y76bi,54.0,0dja032dfcct6e8,1.0,1.0,1.703297,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,1.526316,0.052632,4.0,zo1ifv9d3ocwksy,734.0,g8e8izd33vphs6r,1.0,124.0,245.750000,1.0
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,152.000000,0.473684,18.0,yv41qqhhwlf2nfg,32.0,16cy2e1d5jl7igv,1.0,1.0,3.666667,1.0
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,6.894737,0.052632,1049.0,zz3d6spn0i68nhz,725.0,04vxqqwddrvyw8j,1.0,1.0,2.061010,1.0
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,1519.736842,459.684211,2.0,vasstdc27m7nks3,3.0,umq82yct2iei4f4,2.0,2.5,2.500000,1.0


In [155]:
b = a.groupby(['bidder_id','auction','url']).size().reset_index()
c = b.groupby(['bidder_id','auction']).size().reset_index()
d = c.groupby(['bidder_id']).mean().reset_index()
d = d.rename(columns = {0:'mean_url_per_auction'})
train_n_test = pd.merge(train_n_test, d, on='bidder_id', how='left')
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,...,min_dt_time_other_ip,total_url,most_used_url,amount_most_used_url,rare_url,amount_rare_url,median_bid_per_url,mean_bid_per_url,median_url_per_auction,mean_url_per_auction
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,...,5.210526,1.0,vasstdc27m7nks3,24.0,vasstdc27m7nks3,24.0,24.0,24.000000,1.0,1.000000
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,...,1989.315789,2.0,vgbf6zjir47a9y8,2.0,vasstdc27m7nks3,1.0,1.5,1.500000,2.0,2.000000
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,...,1450.842105,2.0,vasstdc27m7nks3,3.0,i7m2pm7a1w52dqd,1.0,2.0,2.000000,1.0,1.000000
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,...,NaN,1.0,vasstdc27m7nks3,1.0,vasstdc27m7nks3,1.0,1.0,1.000000,1.0,1.000000
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,...,0.052632,91.0,zui5a6sfx0y76bi,54.0,0dja032dfcct6e8,1.0,1.0,1.703297,2.0,4.695652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,...,0.052632,4.0,zo1ifv9d3ocwksy,734.0,g8e8izd33vphs6r,1.0,124.0,245.750000,1.0,1.292683
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0,1.0,2.062500,...,0.473684,18.0,yv41qqhhwlf2nfg,32.0,16cy2e1d5jl7igv,1.0,1.0,3.666667,1.0,1.406250
6710,4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,NaN,2162.0,419.0,115.0,2.0,5.159905,...,0.052632,1049.0,zz3d6spn0i68nhz,725.0,04vxqqwddrvyw8j,1.0,1.0,2.061010,1.0,3.921241
6711,4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,NaN,5.0,5.0,1.0,1.0,1.000000,...,459.684211,2.0,vasstdc27m7nks3,3.0,umq82yct2iei4f4,2.0,2.5,2.500000,1.0,1.000000


In [230]:
train_n_test

,index,bidder_id,payment_account,address,outcome,total_bid,total_auction,max_bid_per_auction,median_bid_per_auction,mean_bid_per_auction,total_device,most_used_device,bid_most_used_device,rare_device,bid_rare_device,median_dt_since_start,median_dt_until_end,median_dt_time_self,min_dt_time_self,median_dt_time_other,min_dt_time_others,total_country,most_freq_country,amount_most_freq_country,rare_country,amount_rare_country,median_country_per_auction,mean_country_per_auction,total_ip,most_used_ip,amount_most_used_ip,rare_ip,amount_rare_ip,median_ip_per_auction,mean_ip_per_auction,median_bid_per_ip,mean_bid_per_ip,ratio_used_unique_ip,median_dt_time_same_ip,min_dt_time_same_ip,median_dt_time_other_ip,min_dt_time_other_ip,total_url,most_used_url,amount_most_used_url,rare_url,amount_rare_url,median_bid_per_url,mean_bid_per_url,median_url_per_auction,mean_url_per_auction
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,18.0,3.0,1.0,1.333333,14.0,phone79,6.0,phone1004,1.0,4489.263158,5956.973684,345.842105,5.210526,8.105263,0.000000,6.0,zm,13.0,et,1.0,1.0,1.166667,20.0,9.194.173.71,4.0,100.75.218.230,1.0,1.0,1.333333,1.0,1.200000,0.250000,0.000000,0.000000,345.842105,5.210526,1.0,vasstdc27m7nks3,24.0,vasstdc27m7nks3,24.0,24.0,24.000000,1.0,1.000000
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,1.0,3.0,3.0,3.000000,2.0,phone293,2.0,phone286,1.0,6793.263158,6848.789474,3233.578947,1989.315789,0.052632,0.000000,1.0,id,3.0,id,3.0,1.0,1.000000,3.0,45.22.158.84,1.0,19.186.111.152,1.0,3.0,3.000000,1.0,1.000000,1.000000,0.000000,0.000000,3233.578947,1989.315789,2.0,vgbf6zjir47a9y8,2.0,vasstdc27m7nks3,1.0,1.5,1.500000,2.0,2.000000
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,4.0,1.0,1.0,1.000000,2.0,phone4,3.0,phone111,1.0,7707.289474,3057.789474,2532.052632,1450.842105,12.236842,4.157895,1.0,id,4.0,id,4.0,1.0,1.000000,4.0,63.67.37.44,1.0,150.248.104.106,1.0,1.0,1.000000,1.0,1.000000,0.000000,0.000000,0.000000,2532.052632,1450.842105,2.0,vasstdc27m7nks3,3.0,i7m2pm7a1w52dqd,1.0,2.0,2.000000,1.0,1.000000
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.000000,1.0,phone58,1.0,phone58,1.0,74481.421053,2774.684211,0.000000,0.000000,17.368421,17.368421,1.0,my,1.0,my,1.0,1.0,1.000000,1.0,101.146.132.199,1.0,101.146.132.199,1.0,1.0,1.000000,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,vasstdc27m7nks3,1.0,vasstdc27m7nks3,1.0,1.0,1.000000,1.0,1.000000
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,23.0,38.0,2.0,6.739130,53.0,phone95,19.0,phone101,1.0,642.736842,10131.263158,13.684211,0.052632,0.315789,0.000000,2.0,us,154.0,in,1.0,1.0,1.000000,123.0,98.111.158.213,8.0,0.119.34.181,1.0,2.0,5.608696,1.0,1.260163,0.477419,2.157895,0.789474,15.473684,0.052632,91.0,zui5a6sfx0y76bi,54.0,0dja032dfcct6e8,1.0,1.0,1.703297,2.0,4.695652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,NaN,983.0,41.0,86.0,14.0,23.975610,9.0,phone786,421.0,phone110,3.0,65660.157895,7231.315789,1.421053,0.052632,0.736842,0.000000,5.0,us,645.0,id,3.0,1.0,1.121951,22.0,95.88.86.44,645.0,127.100.35.36,1.0,1.0,2.170732,5.0,44.681818,0.003052,1.421053,0.052632,1.526316,0.052632,4.0,zo1ifv9d3ocwksy,734.0,g8e8izd33vphs6r,1.0,124.0,245.750000,1.0,1.292683
6709,4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,NaN,66.0,32.0,16.0

## Filling Missing Values
- fill missing values of numerical features with median
- fill missing values of categorical features with mode
- fill missing values of delta time features with 0

In [168]:
## Filling Missing Values
#train_n_test['total_bid'].isnull().sum() #99
#train_n_test['total_auction'].isnull().sum() #99
#train_n_test['max_bid_per_auction'].isnull().sum() #99
#train_n_test['median_bid_per_auction'].isnull().sum() #99
#train_n_test['mean_bid_per_auction'].isnull().sum() #99

99

In [173]:
train_n_test['total_bid'] = train_n_test['total_bid'].fillna(train_n_test['total_bid'].median())

In [177]:
train_n_test['total_auction'] = train_n_test['total_auction'].fillna(train_n_test['total_auction'].median())

In [180]:
train_n_test['max_bid_per_auction'] = train_n_test['max_bid_per_auction'].fillna(train_n_test['max_bid_per_auction'].median())

In [181]:
train_n_test['median_bid_per_auction'] = train_n_test['median_bid_per_auction'].fillna(train_n_test['median_bid_per_auction'].median())

In [182]:
train_n_test['mean_bid_per_auction'] = train_n_test['mean_bid_per_auction'].fillna(train_n_test['mean_bid_per_auction'].median())

In [188]:
#train_n_test['total_device'].isnull().sum() #99
#train_n_test['most_used_device'].isnull().sum() #99
#train_n_test['bid_most_used_device'].isnull().sum() #99
#train_n_test['rare_device'].isnull().sum() #99
#train_n_test['bid_rare_device'].isnull().sum() #99

99

In [190]:
train_n_test['total_device'] = train_n_test['total_device'].fillna(train_n_test['total_device'].median())
train_n_test['most_used_device'] = train_n_test['most_used_device'].fillna(train_n_test['most_used_device'].mode()[0])
train_n_test['bid_most_used_device'] = train_n_test['bid_most_used_device'].fillna(train_n_test['bid_most_used_device'].median())
train_n_test['rare_device'] = train_n_test['rare_device'].fillna(train_n_test['rare_device'].mode()[0])
train_n_test['bid_rare_device'] = train_n_test['bid_rare_device'].fillna(train_n_test['bid_rare_device'].median())

In [194]:
#train_n_test.iloc[:, 15:21].isnull().sum()

median_dt_since_start      99
median_dt_until_end        99
median_dt_time_self      1157
min_dt_time_self         1157
median_dt_time_other      103
min_dt_time_others        103
dtype: int64

In [195]:
train_n_test.iloc[:, 15:21] = train_n_test.iloc[:, 15:21].fillna(0)

In [196]:
#train_n_test.iloc[:, 15:21].isnull().sum()

median_dt_since_start    0
median_dt_until_end      0
median_dt_time_self      0
min_dt_time_self         0
median_dt_time_other     0
min_dt_time_others       0
dtype: int64

In [197]:
train_n_test['total_country'] = train_n_test['total_country'].fillna(train_n_test['total_country'].median())
train_n_test['most_freq_country'] = train_n_test['most_freq_country'].fillna(train_n_test['most_freq_country'].mode()[0])
train_n_test['amount_most_freq_country'] = train_n_test['amount_most_freq_country'].fillna(train_n_test['amount_most_freq_country'].median())
train_n_test['rare_country'] = train_n_test['rare_country'].fillna(train_n_test['rare_country'].mode()[0])
train_n_test['amount_rare_country'] = train_n_test['amount_rare_country'].fillna(train_n_test['amount_rare_country'].median())

In [198]:
train_n_test['median_country_per_auction'] = train_n_test['median_country_per_auction'].fillna(train_n_test['median_country_per_auction'].median())
train_n_test['mean_country_per_auction'] = train_n_test['mean_country_per_auction'].fillna(train_n_test['mean_country_per_auction'].median())

In [199]:
train_n_test['total_ip'] = train_n_test['total_ip'].fillna(train_n_test['total_ip'].median())
train_n_test['most_used_ip'] = train_n_test['most_used_ip'].fillna(train_n_test['most_used_ip'].mode()[0])
train_n_test['amount_most_used_ip'] = train_n_test['amount_most_used_ip'].fillna(train_n_test['amount_most_used_ip'].median())
train_n_test['rare_ip'] = train_n_test['rare_ip'].fillna(train_n_test['rare_ip'].mode()[0])
train_n_test['amount_rare_ip'] = train_n_test['amount_rare_ip'].fillna(train_n_test['amount_rare_ip'].median())

In [202]:
train_n_test.iloc[:, 33:37] = train_n_test.iloc[:, 33:37].fillna(train_n_test.iloc[:, 33:37].median())

In [207]:
#train_n_test.iloc[:, 37:42].isnull().sum()

median_dt_time_same_ip     2722
min_dt_time_same_ip        2722
median_dt_time_other_ip    1337
min_dt_time_other_ip       1337
dtype: int64

In [206]:
train_n_test['ratio_used_unique_ip'] = train_n_test['ratio_used_unique_ip'].fillna(train_n_test['ratio_used_unique_ip'].median())

In [210]:
train_n_test.iloc[:, 38:42] = train_n_test.iloc[:, 38:42].fillna(0)

In [212]:
train_n_test['total_url'] = train_n_test['total_url'].fillna(train_n_test['total_url'].median())
train_n_test['most_used_url'] = train_n_test['most_used_url'].fillna(train_n_test['most_used_url'].mode()[0])
train_n_test['amount_most_used_url'] = train_n_test['amount_most_used_url'].fillna(train_n_test['amount_most_used_url'].median())
train_n_test['rare_url'] = train_n_test['rare_url'].fillna(train_n_test['rare_url'].mode()[0])
train_n_test['amount_rare_url'] = train_n_test['amount_rare_url'].fillna(train_n_test['amount_rare_url'].median())

In [222]:
train_n_test.iloc[:, 47:] = train_n_test.iloc[:, 47:].fillna(train_n_test.iloc[:, 47:].median())

In [242]:
train_n_test.isnull().sum()

index                            0
bidder_id                        0
payment_account                  0
address                          0
outcome                       4700
total_bid                        0
total_auction                    0
max_bid_per_auction              0
median_bid_per_auction           0
mean_bid_per_auction             0
total_device                     0
most_used_device                 0
bid_most_used_device             0
rare_device                      0
bid_rare_device                  0
median_dt_since_start            0
median_dt_until_end              0
median_dt_time_self              0
min_dt_time_self                 0
median_dt_time_other             0
min_dt_time_others               0
total_country                    0
most_freq_country                0
amount_most_freq_country         0
rare_country                     0
amount_rare_country              0
median_country_per_auction       0
mean_country_per_auction         0
total_ip            

In [244]:
train_n_test['rare_url'].describe()

count                6713
unique               5368
top       vasstdc27m7nks3
freq                 1192
Name: rare_url, dtype: object

In [245]:
## Train, test split
train_n_test = train_n_test.drop(['index','bidder_id','address','payment_account','most_used_ip','rare_ip','most_used_url','rare_url'], axis=1)

In [256]:
train_n_test = pd.get_dummies(train_n_test)

In [257]:
train_n_test

outcome  total_bid  total_auction  max_bid_per_auction  \
0         0.0       24.0           18.0                  3.0   
1         0.0        3.0            1.0                  3.0   
2         0.0        4.0            4.0                  1.0   
3         0.0        1.0            1.0                  1.0   
4         0.0      155.0           23.0                 38.0   
...       ...        ...            ...                  ...   
6708      NaN      983.0           41.0                 86.0   
6709      NaN       66.0           32.0                 16.0   
6710      NaN     2162.0          419.0                115.0   
6711      NaN        5.0            5.0                  1.0   
6712      NaN      382.0          116.0                 15.0   

      median_bid_per_auction  mean_bid_per_auction  total_device  \
0                        1.0              1.333333          14.0   
1                        3.0              3.000000           2.0   
2                        1.0              1.000000           2.0   
3                        1.0              1.000000           1.0   
4                        2.0              6.739130          53.0   
...                      ...                   ...           ...   
6708                    14.0             23.975610           9.0   
6709                     1.0              2.062500          29.0   
6710                     2.0              5.159905         376.0   
6711                     1.0              1.000000           4.0   
6712                     2.0              3.293103         117.0   

      bid_most_used_device  bid_rare_device  median_dt_since_start  \
0                      6.0              1.0            4489.263158   
1                      2.0              1.0            6793.263158   
2                      3.0              1.0            7707.289474   
3                      1.0              1.0           74481.421053   
4                     19.0              1.0             642.736842   
...                    ...              ...                    ...   
6708                 421.0              3.0           65660.157895   
6709                  22.0              1.0            5046.789474   
6710                 349.0              1.0            8635.315789   
6711                   2.0              1.0            4494.473684   
6712                  39.0              1.0            8295.894737   

      median_dt_until_end  median_dt_time_self  min_dt_time_self  \
0             5956.973684           345.842105          5.210526   
1             6848.789474          3233.578947       1989.315789   
2             3057.789474          2532.052632       1450.842105   
3             2774.684211             0.000000          0.000000   
4            10131.263158            13.684211          0.052632   
...                   ...                  ...               ...   
6708          7231.315789             1.421053          0.052632   
6709          6678.578947            95.842105          0.473684   
6710         53643.578947             6.578947          0.052632   
6711         71239.263158          1519.736842        459.684211   
6712         63862.210526            35.052632          0.157895   

      median_dt_time_other  min_dt_time_others  total_country  \
0                 8.105263            0.000000            6.0   
1                 0.052632            0.000000            1.0   
2                12.236842            4.157895            1.0   
3                17.368421           17.368421            1.0   
4                 0.315789            0.000000            2.0   
...                    ...                 ...            ...   
6708              0.736842            0.000000            5.0   
6709              2.000000            0.000000           10.0   
6710              3.526316            0.000000           86.0   
6711              1.000000            0.526316            1.0   
6712              2.368421            0.000000 

In [258]:
X_train = train_n_test.iloc[:len(train), 1:]
Y_train = train_n_test.iloc[:len(train), 0]
X_test = train_n_test.iloc[len(train):, 1:]

## Modelling
Using combination of 5 powerful models:
- Logistic Regression
- Random Forest
- Gradient Boosting Tree
- XGBoost
- LightGBM

### Logistic Regression
- CV accuracy mean = 0.945845 (on train set)

In [279]:
# Logistic Regression
logistic = make_pipeline(RobustScaler(), LogisticRegression(random_state=1, max_iter=100))
logistic.fit(X_train, Y_train)
kf = KFold(n_splits=10, shuffle=True, random_state=7).get_n_splits(X_train)
accuracy_logistic = cross_val_score(estimator = logistic, X = X_train, y = Y_train, cv = kf)
accuracy_logistic.mean()

D:\Programs\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
D:\Programs\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regress

0.9458450322644205

### Random Forest
- CV accuracy mean = 0.953797 (on train set)

In [278]:
# Random Forest
rf = RandomForestClassifier(n_estimators=800, criterion='entropy', random_state=1)
rf.fit(X_train, Y_train)
kf = KFold(n_splits=10, shuffle=True, random_state=7).get_n_splits(X_train)
accuracy_rf = cross_val_score(estimator = rf, X = X_train, y = Y_train, cv = kf)
accuracy_rf.mean()

0.9537978424708143

### Gradient Boosting Tree
- CV accuracy mean = 0.953297 (on train set)

In [283]:
# Gradient Boosting
gb = GradientBoostingClassifier(n_estimators=800, loss='deviance', learning_rate=0.01, max_depth=5, random_state=1)
gb.fit(X_train, Y_train)
kf = KFold(n_splits=10, shuffle=True, random_state=7).get_n_splits(X_train)
accuracy_gb = cross_val_score(estimator = gb, X = X_train, y = Y_train, cv = kf)
accuracy_gb.mean()

0.9532978671001426

### XGBoost
- CV accuracy mean = 0.957268 (on train set)

In [286]:
xgboost = xgb.XGBClassifier(n_jobs=-1, learning_rate=0.05, gamma=0.05, 
                            subsample=0.5, max_depth=5, objective='binary:logistic', 
                            base_score=0.5, n_estimators=800)
xgboost.fit(X_train, Y_train)
kf = KFold(n_splits=10, shuffle=True, random_state=7).get_n_splits(X_train)
accuracy_xgb = cross_val_score(estimator = xgboost, X = X_train, y = Y_train, cv = kf)
accuracy_xgb.mean()

0.9572681148711885

### LightGBM
- CV accuracy mean = 0.957270 (on train set)

In [290]:
lgbm = lgb.LGBMClassifier(n_estimators=800, objective='binary', learning_rate=0.05, 
                          max_depth=5, random_state=1, reg_lambda=0.75, subsample=0.5, 
                          n_jobs=-1)
lgbm.fit(X_train, Y_train)
kf = KFold(n_splits=10, shuffle=True, random_state=7).get_n_splits(X_train)
accuracy_lgb = cross_val_score(estimator = lgbm, X = X_train, y = Y_train, cv = kf)
accuracy_lgb.mean()

0.9572705778040491

## Test Prediction
- Combining the probability prediction of 5 models to the final prediction

In [291]:
## Prediction
pred_Y_logistic = logistic.predict_proba(X_test)

In [293]:
pred_Y_rf = rf.predict_proba(X_test)

In [295]:
pred_Y_gb = gb.predict_proba(X_test)

In [297]:
pred_Y_xgb = xgboost.predict_proba(X_test)

In [304]:
pred_Y_lgb = lgbm.predict_proba(X_test)

In [306]:
pred_Y_lgb[:,1]

array([3.12634961e-06, 8.49993143e-06, 4.09252854e-06, ...,
       2.24267048e-03, 4.34341116e-06, 2.03612436e-04])

In [314]:
estimator = {'1':pred_Y_logistic[:,1], '2':pred_Y_rf[:,1], '3':pred_Y_gb[:,1], '4':pred_Y_xgb[:,1], '5':pred_Y_lgb[:,1]}

In [316]:
output = pd.DataFrame(data=estimator)

In [321]:
output['prediction'] = (output['1'] + output['2'] + output['3'] + output['4'] + output['5']) / 5

In [325]:
output = output.drop(['1','2','3','4','5'], axis=1)

In [326]:
output

,prediction
0,0.000458
1,0.001236
2,0.084373
3,0.036008
4,0.005627
...,...
4695,0.706398
4696,0.041260
4697,0.031106
4698,0.000441


## Make Submission csv file
- make the final submission file includes bidder id and prediction

In [327]:
output_id = test['bidder_id']

In [330]:
output_id = pd.DataFrame(data=output_id)

In [331]:
output_id

,bidder_id
0,49bb5a3c944b8fc337981cc7a9ccae41u31d7
1,a921612b85a1494456e74c09393ccb65ylp4y
2,6b601e72a4d264dab9ace9d7b229b47479v6i
3,eaf0ed0afc9689779417274b4791726cn5udi
4,cdecd8d02ed8c6037e38042c7745f688mx5sf
...,...
4695,bef56983ba78b2ee064443ae95972877jfkyd
4696,4da45cc915c32d4368ac7e773d92d4affwqrr
4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9
4698,4981c32c54dde65b79dbc48fd9ab6457caqze


In [333]:
submission = pd.merge(output_id, output, left_index=True, right_index=True)

In [334]:
submission

,bidder_id,prediction
0,49bb5a3c944b8fc337981cc7a9ccae41u31d7,0.000458
1,a921612b85a1494456e74c09393ccb65ylp4y,0.001236
2,6b601e72a4d264dab9ace9d7b229b47479v6i,0.084373
3,eaf0ed0afc9689779417274b4791726cn5udi,0.036008
4,cdecd8d02ed8c6037e38042c7745f688mx5sf,0.005627
...,...,...
4695,bef56983ba78b2ee064443ae95972877jfkyd,0.706398
4696,4da45cc915c32d4368ac7e773d92d4affwqrr,0.041260
4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,0.031106
4698,4981c32c54dde65b79dbc48fd9ab6457caqze,0.000441


In [337]:
submission.to_csv('facebook_auction_submission.csv', index=False, header=True)